In [1]:
import requests
import json
from pprint import pprint
import pandas as pd
import time


#import spotify keys
from config import client_id
from config import client_secret

# Creating Access Token for Spotify API

In [80]:
#generate access token for spotify api
#from: https://stmorse.github.io/journal/spotify-api.html

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [81]:
#from: https://stmorse.github.io/journal/spotify-api.html

#base URL for Spotify API search
search_url = 'https://api.spotify.com/v1/search'

#In order to access, we send a properly formed GET request to the API server, with our access_token in the header. Let’s save this header info now, in the following very specific format:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# Creating CSVs of Regional Top 200 Charts and Global Top 200 Chart

In [4]:
#import regional top songs csvs and save as individual dfs
regional_files = {"global":"regional-global-weekly-2022-10-06.csv", "vietnam":"regional-vietnam-weekly-2022-10-06.csv","venezuela":"regional-venezuela-weekly-2022-10-06.csv",
"uruguay":"regional-uruguay-weekly-2022-10-06.csv","united_kingdom":"regional-united_kingdom-weekly-2022-10-06.csv","ukraine":"regional-ukraine-weekly-2022-10-06.csv",
"uae":"regional-uae-weekly-2022-10-06.csv","turkey":"regional-turkey-weekly-2022-10-06.csv","thailand":"regional-thailand-weekly-2022-10-06.csv",
"taiwan":"regional-taiwan-weekly-2022-10-06.csv","switzerland":"regional-switzerland-weekly-2022-10-06.csv","sweden":"regional-sweden-weekly-2022-10-06.csv",
"spain":"regional-spain-weekly-2022-10-06.csv","south_korea":"regional-south_korea-weekly-2022-10-06.csv","south_africa":"regional-south_africa-weekly-2022-10-06.csv",
"slovakia":"regional-slovakia-weekly-2022-10-06.csv","singapore":"regional-singapore-weekly-2022-10-06.csv","saudi_arabia":"regional-saudi_arabia-weekly-2022-10-06.csv",
"romania":"regional-romania-weekly-2022-10-06.csv","portugal":"regional-portugal-weekly-2022-10-06.csv","poland":"regional-poland-weekly-2022-10-06.csv",
"philippines":"regional-philippines-weekly-2022-10-06.csv","peru":"regional-peru-weekly-2022-10-06.csv","paraguay":"regional-paraguay-weekly-2022-10-06.csv",
"panama":"regional-panama-weekly-2022-10-06.csv","pakistan":"regional-pakistan-weekly-2022-10-06.csv","norway":"regional-norway-weekly-2022-10-06.csv",
"nigeria":"regional-nigeria-weekly-2022-10-06.csv","nicaragua":"regional-nicaragua-weekly-2022-10-06.csv","new_zealand":"regional-new_zealand-weekly-2022-10-06.csv",
"netherlands":"regional-netherlands-weekly-2022-10-06.csv","morocco":"regional-morocco-weekly-2022-10-06.csv","mexico":"regional-mexico-weekly-2022-10-06.csv",
"malaysia":"regional-malaysia-weekly-2022-10-06.csv","luxembourg":"regional-luxembourg-weekly-2022-10-06.csv","lithuania":"regional-lithuania-weekly-2022-10-06.csv",
"latvia":"regional-latvia-weekly-2022-10-06.csv","kazakhstan":"regional-kazakhstan-weekly-2022-10-06.csv","japan":"regional-japan-weekly-2022-10-06.csv",
"italy":"regional-italy-weekly-2022-10-06.csv","israel":"regional-israel-weekly-2022-10-06.csv","ireland":"regional-ireland-weekly-2022-10-06.csv",
"indonesia":"regional-indonesia-weekly-2022-10-06.csv","india":"regional-india-weekly-2022-10-06.csv","iceland":"regional-iceland-weekly-2022-10-06.csv",
"hungary":"regional-hungary-weekly-2022-10-06.csv","hong_kong":"regional-hong_kong-weekly-2022-10-06.csv","honduras":"regional-honduras-weekly-2022-10-06.csv",
"guatemala":"regional-guatemala-weekly-2022-10-06.csv","greece":"regional-greece-weekly-2022-10-06.csv","germany":"regional-germany-weekly-2022-10-06.csv",
"france":"regional-france-weekly-2022-10-06.csv","finland":"regional-finland-weekly-2022-10-06.csv","estonia":"regional-estonia-weekly-2022-10-06.csv",
"el_salvador":"regional-el_salvador-weekly-2022-10-06.csv","egypt":"regional-egypt-weekly-2022-10-06.csv","ecuador":"regional-ecuador-weekly-2022-10-06.csv",
"dominican_republic":"regional-dominican_republic-weekly-2022-10-06.csv","denmark":"regional-denmark-weekly-2022-10-06.csv","czech_republic":"regional-czech_republic-weekly-2022-10-06.csv",
"cyprus":"regional-cyprus-weekly-2022-10-06.csv","costa_rica":"regional-costa_rica-weekly-2022-10-06.csv","colombia":"regional-colombia-weekly-2022-10-06.csv",
"chile":"regional-chile-weekly-2022-10-06.csv","canada":"regional-canada-weekly-2022-10-06.csv","bulgaria":"regional-bulgaria-weekly-2022-10-06.csv",
"brazil":"regional-brazil-weekly-2022-10-06.csv","bolivia":"regional-bolivia-weekly-2022-10-06.csv","belgium":"regional-belgium-weekly-2022-10-06.csv",
"belarus":"regional-belarus-weekly-2022-10-06.csv","austria":"regional-austria-weekly-2022-10-06.csv","australia":"regional-australia-weekly-2022-10-06.csv",
"argentina":"regional-argentina-weekly-2022-10-06.csv"}

d = {}

for key in regional_files:
    # print(key, regional_files[key])
    d[key] = pd.read_csv(f'resources/top_chart_downloads/{regional_files[key]}') 
    
    #extracting track id (removing "spotify:track:" from track id in csv)
    track_id_series = d[key].uri.str[14:]
    
    #convert to df and rename column
    track_id_df = track_id_series.to_frame().rename(columns={"uri":"track_id"})
    
    #merge dfs
    d[key] = d[key].merge(track_id_df, how="inner", left_index=True, right_index=True)
    d[key] = d[key][["track_id", "artist_names", "track_name"]]
    #add column for region
    d[key]['region'] = key.capitalize()
    #convert values to string
    d[key] =d[key].astype({'track_id':'string','artist_names':'string','track_name':'string','region':'string'})

In [5]:
# test import of csv chart
d['argentina']

,track_id,artist_names,track_name,region
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Argentina
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Argentina
2,45dlaEXrXW47TVbrQ4gK7E,"Rei, Callejero Fino",Tu Turrito,Argentina
3,3Ne5uYDFzHEFYAPbaBMIck,Maria Becerra,AUTOMÁTICO,Argentina
4,53tfEupEzQRtVFOeZvk7xq,ROSALÍA,DESPECHÁ,Argentina
...,...,...,...,...
195,6nAlJuLlRlpAcq30xIqKJs,Plan B,¿Por Que Te Demoras?,Argentina
196,4iaNmAvcZ6sPGvgZ6vQQdx,"Quevedo, Linton",Ahora y Siempre,Argentina
197,65DBZofI0b79kfHTcWWDuU,Soda Stereo,Trátame Suavemente - Remasterizado 2007,Argentina
198,0WuKq2LiraBFney78dzwoc,Ciro y los Persas,Mírenla,Argentina


In [6]:
charts_list = d.keys()
charts_list

dict_keys(['global', 'vietnam', 'venezuela', 'uruguay', 'united_kingdom', 'ukraine', 'uae', 'turkey', 'thailand', 'taiwan', 'switzerland', 'sweden', 'spain', 'south_korea', 'south_africa', 'slovakia', 'singapore', 'saudi_arabia', 'romania', 'portugal', 'poland', 'philippines', 'peru', 'paraguay', 'panama', 'pakistan', 'norway', 'nigeria', 'nicaragua', 'new_zealand', 'netherlands', 'morocco', 'mexico', 'malaysia', 'luxembourg', 'lithuania', 'latvia', 'kazakhstan', 'japan', 'italy', 'israel', 'ireland', 'indonesia', 'india', 'iceland', 'hungary', 'hong_kong', 'honduras', 'guatemala', 'greece', 'germany', 'france', 'finland', 'estonia', 'el_salvador', 'egypt', 'ecuador', 'dominican_republic', 'denmark', 'czech_republic', 'cyprus', 'costa_rica', 'colombia', 'chile', 'canada', 'bulgaria', 'brazil', 'bolivia', 'belgium', 'belarus', 'austria', 'australia', 'argentina'])

In [7]:
len(charts_list)

73

In [8]:
#base URL for Spotify API
base_url = 'https://api.spotify.com/v1/'

# Argentina Top Chart Track Features

In [9]:
#empty lists to store song data
argentina_danceability = []
argentina_energy = []
argentina_key = []
argentina_loudness = []
argentina_mode = []
argentina_speechiness = []
argentina_acousticness = []
argentina_instrumentalness = []
argentina_liveness = []
argentina_valence = []
argentina_tempo = []
argentina_id_num = []
argentina_duration_mins = []
argentina_time_signature = []

for i in d['argentina']["track_id"]:
    argentina_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    argentina_danceability.append(argentina_response["danceability"])
    argentina_energy.append(argentina_response["energy"])
    argentina_key.append(argentina_response["key"])
    argentina_loudness.append(argentina_response["loudness"])
    argentina_mode.append(argentina_response["mode"])
    argentina_speechiness.append(argentina_response["speechiness"])
    argentina_acousticness.append(argentina_response["acousticness"])
    argentina_instrumentalness.append(argentina_response["instrumentalness"])
    argentina_liveness.append(argentina_response["liveness"])
    argentina_valence.append(argentina_response["valence"])
    argentina_tempo.append(argentina_response["tempo"])
    argentina_id_num.append(argentina_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    argentina_duration_mins.append(round((argentina_response["duration_ms"] / 60000),2))
    argentina_time_signature.append(argentina_response["time_signature"])  

    # time.sleep(1)

argentina_dict = {}
#create a dictionary to hold data gathered from the api
argentina_dict = {
    "track_id": argentina_id_num,
    "danceability": argentina_danceability,
    "energy": argentina_energy,
    "key": argentina_key,
    "loudness": argentina_loudness,
    "mode": argentina_mode,
    "speechiness": argentina_speechiness,
    "acousticness": argentina_acousticness,
    "instrumentalness": argentina_instrumentalness,
    "liveness": argentina_liveness,
    "valence": argentina_valence,
    "tempo": argentina_tempo,
    "duration_mins": argentina_duration_mins,
    "time_signature": argentina_time_signature
}

#convert dictionary to a dataframe
argentina_info_df = pd.DataFrame(argentina_dict)
argentina_info_df.head()

,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
1,2tTmW7RDtMQtBk7m2rYeSw,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
2,45dlaEXrXW47TVbrQ4gK7E,0.838,0.545,9,-6.100,0,0.0604,0.0966,0.000000,0.0927,0.428,90.023,2.75,4
3,3Ne5uYDFzHEFYAPbaBMIck,0.680,0.808,10,-3.854,0,0.2240,0.0911,0.000002,0.2050,0.868,176.131,2.92,4
4,53tfEupEzQRtVFOeZvk7xq,0.914,0.622,7,-6.538,1,0.0892,0.1460,0.000017,0.0632,0.770,130.040,2.61,4


In [10]:
argentina_tracks = d['argentina']
argentina_df = argentina_tracks.merge(argentina_info_df, how="inner")
argentina_df.head()

,track_id,artist_names,track_name,region,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_mins,time_signature
0,5ww2BF9slyYgNOk37BlC4u,Manuel Turizo,La Bachata,Argentina,0.835,0.679,7,-5.329,0,0.0364,0.5830,0.000002,0.2180,0.850,124.980,2.71,4
1,2tTmW7RDtMQtBk7m2rYeSw,"Bizarrap, Quevedo","Quevedo: Bzrp Music Sessions, Vol. 52",Argentina,0.621,0.782,2,-5.548,1,0.0440,0.0125,0.033000,0.2300,0.550,128.033,3.32,4
2,45dlaEXrXW47TVbrQ4gK7E,"Rei, Callejero Fino",Tu Turrito,Argentina,0.838,0.545,9,-6.100,0,0.0604,0.0966,0.000000,0.0927,0.428,90.023,2.75,4
3,3Ne5uYDFzHEFYAPbaBMIck,Maria Becerra,AUTOMÁTICO,Argentina,0.680,0.808,10,-3.854,0,0.2240,0.0911,0.000002,0.2050,0.868,176.131,2.92,4
4,53tfEupEzQRtVFOeZvk7xq,ROSALÍA,DESPECHÁ,Argentina,0.914,0.622,7,-6.538,1,0.0892,0.1460,0.000017,0.0632,0.770,130.040,2.61,4


In [11]:
argentina_df.to_csv('resources/top_chart_features/argentina_df.csv', index=False)

# Australia Top Chart Track Features

In [12]:
#empty lists to store song data
australia_danceability = []
australia_energy = []
australia_key = []
australia_loudness = []
australia_mode = []
australia_speechiness = []
australia_acousticness = []
australia_instrumentalness = []
australia_liveness = []
australia_valence = []
australia_tempo = []
australia_id_num = []
australia_duration_mins = []
australia_time_signature = []

for i in d['australia']["track_id"]:
    australia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    australia_danceability.append(australia_response["danceability"])
    australia_energy.append(australia_response["energy"])
    australia_key.append(australia_response["key"])
    australia_loudness.append(australia_response["loudness"])
    australia_mode.append(australia_response["mode"])
    australia_speechiness.append(australia_response["speechiness"])
    australia_acousticness.append(australia_response["acousticness"])
    australia_instrumentalness.append(australia_response["instrumentalness"])
    australia_liveness.append(australia_response["liveness"])
    australia_valence.append(australia_response["valence"])
    australia_tempo.append(australia_response["tempo"])
    australia_id_num.append(australia_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    australia_duration_mins.append(round((australia_response["duration_ms"] / 60000),2))
    australia_time_signature.append(australia_response["time_signature"])  

    # time.sleep(1)

australia_dict = {}
#create a dictionary to hold data gathered from the api
australia_dict = {
    "track_id": australia_id_num,
    "danceability": australia_danceability,
    "energy": australia_energy,
    "key": australia_key,
    "loudness": australia_loudness,
    "mode": australia_mode,
    "speechiness": australia_speechiness,
    "acousticness": australia_acousticness,
    "instrumentalness": australia_instrumentalness,
    "liveness": australia_liveness,
    "valence": australia_valence,
    "tempo": australia_tempo,
    "duration_mins": australia_duration_mins,
    "time_signature": australia_time_signature
}

#convert dictionary to a dataframe
australia_info_df = pd.DataFrame(australia_dict)

australia_tracks = d['australia']
australia_df = australia_tracks.merge(australia_info_df, how="inner")
australia_df.to_csv('resources/top_chart_features/australia_df.csv', index=False)

# Austria Top Char# Austria Top Chart Track Featurest Track Features

In [13]:
#empty lists to store song data
austria_danceability = []
austria_energy = []
austria_key = []
austria_loudness = []
austria_mode = []
austria_speechiness = []
austria_acousticness = []
austria_instrumentalness = []
austria_liveness = []
austria_valence = []
austria_tempo = []
austria_id_num = []
austria_duration_mins = []
austria_time_signature = []

for i in d['austria']["track_id"]:
    austria_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    austria_danceability.append(austria_response["danceability"])
    austria_energy.append(austria_response["energy"])
    austria_key.append(austria_response["key"])
    austria_loudness.append(austria_response["loudness"])
    austria_mode.append(austria_response["mode"])
    austria_speechiness.append(austria_response["speechiness"])
    austria_acousticness.append(austria_response["acousticness"])
    austria_instrumentalness.append(austria_response["instrumentalness"])
    austria_liveness.append(austria_response["liveness"])
    austria_valence.append(austria_response["valence"])
    austria_tempo.append(austria_response["tempo"])
    austria_id_num.append(austria_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    austria_duration_mins.append(round((austria_response["duration_ms"] / 60000),2))
    austria_time_signature.append(austria_response["time_signature"])  

    # time.sleep(1)

austria_dict = {}
#create a dictionary to hold data gathered from the api
austria_dict = {
    "track_id": austria_id_num,
    "danceability": austria_danceability,
    "energy": austria_energy,
    "key": austria_key,
    "loudness": austria_loudness,
    "mode": austria_mode,
    "speechiness": austria_speechiness,
    "acousticness": austria_acousticness,
    "instrumentalness": austria_instrumentalness,
    "liveness": austria_liveness,
    "valence": austria_valence,
    "tempo": austria_tempo,
    "duration_mins": austria_duration_mins,
    "time_signature": austria_time_signature
}

#convert dictionary to a dataframe
austria_info_df = pd.DataFrame(austria_dict)

austria_tracks = d['austria']
austria_df = austria_tracks.merge(austria_info_df, how="inner")
austria_df.to_csv('resources/top_chart_features/austria_df.csv', index=False)

# Belarus Top Chart Track Features

In [14]:
#empty lists to store song data
belarus_danceability = []
belarus_energy = []
belarus_key = []
belarus_loudness = []
belarus_mode = []
belarus_speechiness = []
belarus_acousticness = []
belarus_instrumentalness = []
belarus_liveness = []
belarus_valence = []
belarus_tempo = []
belarus_id_num = []
belarus_duration_mins = []
belarus_time_signature = []

for i in d['belarus']["track_id"]:
    belarus_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    belarus_danceability.append(belarus_response["danceability"])
    belarus_energy.append(belarus_response["energy"])
    belarus_key.append(belarus_response["key"])
    belarus_loudness.append(belarus_response["loudness"])
    belarus_mode.append(belarus_response["mode"])
    belarus_speechiness.append(belarus_response["speechiness"])
    belarus_acousticness.append(belarus_response["acousticness"])
    belarus_instrumentalness.append(belarus_response["instrumentalness"])
    belarus_liveness.append(belarus_response["liveness"])
    belarus_valence.append(belarus_response["valence"])
    belarus_tempo.append(belarus_response["tempo"])
    belarus_id_num.append(belarus_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    belarus_duration_mins.append(round((belarus_response["duration_ms"] / 60000),2))
    belarus_time_signature.append(belarus_response["time_signature"])  

    # time.sleep(1)

belarus_dict = {}
#create a dictionary to hold data gathered from the api
belarus_dict = {
    "track_id": belarus_id_num,
    "danceability": belarus_danceability,
    "energy": belarus_energy,
    "key": belarus_key,
    "loudness": belarus_loudness,
    "mode": belarus_mode,
    "speechiness": belarus_speechiness,
    "acousticness": belarus_acousticness,
    "instrumentalness": belarus_instrumentalness,
    "liveness": belarus_liveness,
    "valence": belarus_valence,
    "tempo": belarus_tempo,
    "duration_mins": belarus_duration_mins,
    "time_signature": belarus_time_signature
}

#convert dictionary to a dataframe
belarus_info_df = pd.DataFrame(belarus_dict)

belarus_tracks = d['belarus']
belarus_df = belarus_info_df.merge(belarus_tracks, how="inner")
belarus_df.to_csv('resources/top_chart_features/belarus_df.csv', index=False)

# Belgium Top Chart Track Features

In [15]:
#empty lists to store song data
belgium_danceability = []
belgium_energy = []
belgium_key = []
belgium_loudness = []
belgium_mode = []
belgium_speechiness = []
belgium_acousticness = []
belgium_instrumentalness = []
belgium_liveness = []
belgium_valence = []
belgium_tempo = []
belgium_id_num = []
belgium_duration_mins = []
belgium_time_signature = []

for i in d['belgium']["track_id"]:
    belgium_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    belgium_danceability.append(belgium_response["danceability"])
    belgium_energy.append(belgium_response["energy"])
    belgium_key.append(belgium_response["key"])
    belgium_loudness.append(belgium_response["loudness"])
    belgium_mode.append(belgium_response["mode"])
    belgium_speechiness.append(belgium_response["speechiness"])
    belgium_acousticness.append(belgium_response["acousticness"])
    belgium_instrumentalness.append(belgium_response["instrumentalness"])
    belgium_liveness.append(belgium_response["liveness"])
    belgium_valence.append(belgium_response["valence"])
    belgium_tempo.append(belgium_response["tempo"])
    belgium_id_num.append(belgium_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    belgium_duration_mins.append(round((belgium_response["duration_ms"] / 60000),2))
    belgium_time_signature.append(belgium_response["time_signature"])  

    # time.sleep(1)

belgium_dict = {}
#create a dictionary to hold data gathered from the api
belgium_dict = {
    "track_id": belgium_id_num,
    "danceability": belgium_danceability,
    "energy": belgium_energy,
    "key": belgium_key,
    "loudness": belgium_loudness,
    "mode": belgium_mode,
    "speechiness": belgium_speechiness,
    "acousticness": belgium_acousticness,
    "instrumentalness": belgium_instrumentalness,
    "liveness": belgium_liveness,
    "valence": belgium_valence,
    "tempo": belgium_tempo,
    "duration_mins": belgium_duration_mins,
    "time_signature": belgium_time_signature
}

#convert dictionary to a dataframe
belgium_info_df = pd.DataFrame(belgium_dict)

belgium_tracks = d['belgium']
belgium_df = belgium_tracks.merge(belgium_info_df, how="inner")
belgium_df.to_csv('resources/top_chart_features/belgium_df.csv', index=False)

# Bolivia Top Chart Track Features

In [16]:
#empty lists to store song data
bolivia_danceability = []
bolivia_energy = []
bolivia_key = []
bolivia_loudness = []
bolivia_mode = []
bolivia_speechiness = []
bolivia_acousticness = []
bolivia_instrumentalness = []
bolivia_liveness = []
bolivia_valence = []
bolivia_tempo = []
bolivia_id_num = []
bolivia_duration_mins = []
bolivia_time_signature = []

for i in d['bolivia']["track_id"]:
    bolivia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    bolivia_danceability.append(bolivia_response["danceability"])
    bolivia_energy.append(bolivia_response["energy"])
    bolivia_key.append(bolivia_response["key"])
    bolivia_loudness.append(bolivia_response["loudness"])
    bolivia_mode.append(bolivia_response["mode"])
    bolivia_speechiness.append(bolivia_response["speechiness"])
    bolivia_acousticness.append(bolivia_response["acousticness"])
    bolivia_instrumentalness.append(bolivia_response["instrumentalness"])
    bolivia_liveness.append(bolivia_response["liveness"])
    bolivia_valence.append(bolivia_response["valence"])
    bolivia_tempo.append(bolivia_response["tempo"])
    bolivia_id_num.append(bolivia_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    bolivia_duration_mins.append(round((bolivia_response["duration_ms"] / 60000),2))
    bolivia_time_signature.append(bolivia_response["time_signature"])  

    # time.sleep(1)

bolivia_dict = {}
#create a dictionary to hold data gathered from the api
bolivia_dict = {
    "track_id": bolivia_id_num,
    "danceability": bolivia_danceability,
    "energy": bolivia_energy,
    "key": bolivia_key,
    "loudness": bolivia_loudness,
    "mode": bolivia_mode,
    "speechiness": bolivia_speechiness,
    "acousticness": bolivia_acousticness,
    "instrumentalness": bolivia_instrumentalness,
    "liveness": bolivia_liveness,
    "valence": bolivia_valence,
    "tempo": bolivia_tempo,
    "duration_mins": bolivia_duration_mins,
    "time_signature": bolivia_time_signature
}

#convert dictionary to a dataframe
bolivia_info_df = pd.DataFrame(bolivia_dict)

bolivia_tracks = d['bolivia']
bolivia_df = bolivia_tracks.merge(bolivia_info_df, how="inner")
bolivia_df.to_csv('resources/top_chart_features/bolivia_df.csv', index=False)

# Brazil Top Chart Track Features

In [17]:
#empty lists to store song data
brazil_danceability = []
brazil_energy = []
brazil_key = []
brazil_loudness = []
brazil_mode = []
brazil_speechiness = []
brazil_acousticness = []
brazil_instrumentalness = []
brazil_liveness = []
brazil_valence = []
brazil_tempo = []
brazil_id_num = []
brazil_duration_mins = []
brazil_time_signature = []

for i in d['brazil']["track_id"]:
    brazil_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    brazil_danceability.append(brazil_response["danceability"])
    brazil_energy.append(brazil_response["energy"])
    brazil_key.append(brazil_response["key"])
    brazil_loudness.append(brazil_response["loudness"])
    brazil_mode.append(brazil_response["mode"])
    brazil_speechiness.append(brazil_response["speechiness"])
    brazil_acousticness.append(brazil_response["acousticness"])
    brazil_instrumentalness.append(brazil_response["instrumentalness"])
    brazil_liveness.append(brazil_response["liveness"])
    brazil_valence.append(brazil_response["valence"])
    brazil_tempo.append(brazil_response["tempo"])
    brazil_id_num.append(brazil_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    brazil_duration_mins.append(round((brazil_response["duration_ms"] / 60000),2))
    brazil_time_signature.append(brazil_response["time_signature"])  

    # time.sleep(1)

brazil_dict = {}
#create a dictionary to hold data gathered from the api
brazil_dict = {
    "track_id": brazil_id_num,
    "danceability": brazil_danceability,
    "energy": brazil_energy,
    "key": brazil_key,
    "loudness": brazil_loudness,
    "mode": brazil_mode,
    "speechiness": brazil_speechiness,
    "acousticness": brazil_acousticness,
    "instrumentalness": brazil_instrumentalness,
    "liveness": brazil_liveness,
    "valence": brazil_valence,
    "tempo": brazil_tempo,
    "duration_mins": brazil_duration_mins,
    "time_signature": brazil_time_signature
}

#convert dictionary to a dataframe
brazil_info_df = pd.DataFrame(brazil_dict)

brazil_tracks = d['brazil']
brazil_df = brazil_tracks.merge(brazil_info_df, how="inner")
brazil_df.to_csv('resources/top_chart_features/brazil_df.csv', index=False)

# Bulgaria Top Chart Track Features

In [18]:
#empty lists to store song data
bulgaria_danceability = []
bulgaria_energy = []
bulgaria_key = []
bulgaria_loudness = []
bulgaria_mode = []
bulgaria_speechiness = []
bulgaria_acousticness = []
bulgaria_instrumentalness = []
bulgaria_liveness = []
bulgaria_valence = []
bulgaria_tempo = []
bulgaria_id_num = []
bulgaria_duration_mins = []
bulgaria_time_signature = []

for i in d['bulgaria']["track_id"]:
    bulgaria_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    bulgaria_danceability.append(bulgaria_response["danceability"])
    bulgaria_energy.append(bulgaria_response["energy"])
    bulgaria_key.append(bulgaria_response["key"])
    bulgaria_loudness.append(bulgaria_response["loudness"])
    bulgaria_mode.append(bulgaria_response["mode"])
    bulgaria_speechiness.append(bulgaria_response["speechiness"])
    bulgaria_acousticness.append(bulgaria_response["acousticness"])
    bulgaria_instrumentalness.append(bulgaria_response["instrumentalness"])
    bulgaria_liveness.append(bulgaria_response["liveness"])
    bulgaria_valence.append(bulgaria_response["valence"])
    bulgaria_tempo.append(bulgaria_response["tempo"])
    bulgaria_id_num.append(bulgaria_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    bulgaria_duration_mins.append(round((bulgaria_response["duration_ms"] / 60000),2))
    bulgaria_time_signature.append(bulgaria_response["time_signature"])  

    # time.sleep(1)

bulgaria_dict = {}
#create a dictionary to hold data gathered from the api
bulgaria_dict = {
    "track_id": bulgaria_id_num,
    "danceability": bulgaria_danceability,
    "energy": bulgaria_energy,
    "key": bulgaria_key,
    "loudness": bulgaria_loudness,
    "mode": bulgaria_mode,
    "speechiness": bulgaria_speechiness,
    "acousticness": bulgaria_acousticness,
    "instrumentalness": bulgaria_instrumentalness,
    "liveness": bulgaria_liveness,
    "valence": bulgaria_valence,
    "tempo": bulgaria_tempo,
    "duration_mins": bulgaria_duration_mins,
    "time_signature": bulgaria_time_signature
}

#convert dictionary to a dataframe
bulgaria_info_df = pd.DataFrame(bulgaria_dict)

bulgaria_tracks = d['bulgaria']
bulgaria_df = bulgaria_tracks.merge(bulgaria_info_df, how="inner")
bulgaria_df.to_csv('resources/top_chart_features/bulgaria_df.csv', index=False)

# Canada Top Chart Track Features

In [19]:
#empty lists to store song data
canada_danceability = []
canada_energy = []
canada_key = []
canada_loudness = []
canada_mode = []
canada_speechiness = []
canada_acousticness = []
canada_instrumentalness = []
canada_liveness = []
canada_valence = []
canada_tempo = []
canada_id_num = []
canada_duration_mins = []
canada_time_signature = []

for i in d['canada']["track_id"]:
    canada_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    canada_danceability.append(canada_response["danceability"])
    canada_energy.append(canada_response["energy"])
    canada_key.append(canada_response["key"])
    canada_loudness.append(canada_response["loudness"])
    canada_mode.append(canada_response["mode"])
    canada_speechiness.append(canada_response["speechiness"])
    canada_acousticness.append(canada_response["acousticness"])
    canada_instrumentalness.append(canada_response["instrumentalness"])
    canada_liveness.append(canada_response["liveness"])
    canada_valence.append(canada_response["valence"])
    canada_tempo.append(canada_response["tempo"])
    canada_id_num.append(canada_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    canada_duration_mins.append(round((canada_response["duration_ms"] / 60000),2))
    canada_time_signature.append(canada_response["time_signature"])  

    # time.sleep(1)

canada_dict = {}
#create a dictionary to hold data gathered from the api
canada_dict = {
    "track_id": canada_id_num,
    "danceability": canada_danceability,
    "energy": canada_energy,
    "key": canada_key,
    "loudness": canada_loudness,
    "mode": canada_mode,
    "speechiness": canada_speechiness,
    "acousticness": canada_acousticness,
    "instrumentalness": canada_instrumentalness,
    "liveness": canada_liveness,
    "valence": canada_valence,
    "tempo": canada_tempo,
    "duration_mins": canada_duration_mins,
    "time_signature": canada_time_signature
}

#convert dictionary to a dataframe
canada_info_df = pd.DataFrame(canada_dict)

canada_tracks = d['canada']
canada_df = canada_tracks.merge(canada_info_df, how="inner")
canada_df.to_csv('resources/top_chart_features/canada_df.csv', index=False)

# Chile Top Chart Track Features

In [20]:
#empty lists to store song data
chile_danceability = []
chile_energy = []
chile_key = []
chile_loudness = []
chile_mode = []
chile_speechiness = []
chile_acousticness = []
chile_instrumentalness = []
chile_liveness = []
chile_valence = []
chile_tempo = []
chile_id_num = []
chile_duration_mins = []
chile_time_signature = []

for i in d['chile']["track_id"]:
    chile_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    chile_danceability.append(chile_response["danceability"])
    chile_energy.append(chile_response["energy"])
    chile_key.append(chile_response["key"])
    chile_loudness.append(chile_response["loudness"])
    chile_mode.append(chile_response["mode"])
    chile_speechiness.append(chile_response["speechiness"])
    chile_acousticness.append(chile_response["acousticness"])
    chile_instrumentalness.append(chile_response["instrumentalness"])
    chile_liveness.append(chile_response["liveness"])
    chile_valence.append(chile_response["valence"])
    chile_tempo.append(chile_response["tempo"])
    chile_id_num.append(chile_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    chile_duration_mins.append(round((chile_response["duration_ms"] / 60000),2))
    chile_time_signature.append(chile_response["time_signature"])  

    # time.sleep(1)

chile_dict = {}
#create a dictionary to hold data gathered from the api
chile_dict = {
    "track_id": chile_id_num,
    "danceability": chile_danceability,
    "energy": chile_energy,
    "key": chile_key,
    "loudness": chile_loudness,
    "mode": chile_mode,
    "speechiness": chile_speechiness,
    "acousticness": chile_acousticness,
    "instrumentalness": chile_instrumentalness,
    "liveness": chile_liveness,
    "valence": chile_valence,
    "tempo": chile_tempo,
    "duration_mins": chile_duration_mins,
    "time_signature": chile_time_signature,
}

#convert dictionary to a dataframe
chile_info_df = pd.DataFrame(chile_dict)

chile_tracks = d['chile']
chile_df = chile_tracks.merge(chile_info_df, how="inner")
chile_df.to_csv('resources/top_chart_features/chile_df.csv', index=False)

# Colombia Top Chart Track Features

In [21]:
#empty lists to store song data
colombia_danceability = []
colombia_energy = []
colombia_key = []
colombia_loudness = []
colombia_mode = []
colombia_speechiness = []
colombia_acousticness = []
colombia_instrumentalness = []
colombia_liveness = []
colombia_valence = []
colombia_tempo = []
colombia_id_num = []
colombia_duration_mins = []
colombia_time_signature = []

for i in d['colombia']["track_id"]:
    colombia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    colombia_danceability.append(colombia_response["danceability"])
    colombia_energy.append(colombia_response["energy"])
    colombia_key.append(colombia_response["key"])
    colombia_loudness.append(colombia_response["loudness"])
    colombia_mode.append(colombia_response["mode"])
    colombia_speechiness.append(colombia_response["speechiness"])
    colombia_acousticness.append(colombia_response["acousticness"])
    colombia_instrumentalness.append(colombia_response["instrumentalness"])
    colombia_liveness.append(colombia_response["liveness"])
    colombia_valence.append(colombia_response["valence"])
    colombia_tempo.append(colombia_response["tempo"])
    colombia_id_num.append(colombia_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    colombia_duration_mins.append(round((colombia_response["duration_ms"] / 60000),2))
    colombia_time_signature.append(colombia_response["time_signature"])  

    # time.sleep(1)

colombia_dict = {}
#create a dictionary to hold data gathered from the api
colombia_dict = {
    "track_id": colombia_id_num,
    "danceability": colombia_danceability,
    "energy": colombia_energy,
    "key": colombia_key,
    "loudness": colombia_loudness,
    "mode": colombia_mode,
    "speechiness": colombia_speechiness,
    "acousticness": colombia_acousticness,
    "instrumentalness": colombia_instrumentalness,
    "liveness": colombia_liveness,
    "valence": colombia_valence,
    "tempo": colombia_tempo,
    "duration_mins": colombia_duration_mins,
    "time_signature": colombia_time_signature,
}

#convert dictionary to a dataframe
colombia_info_df = pd.DataFrame(colombia_dict)

colombia_tracks = d['colombia']
colombia_df = colombia_tracks.merge(colombia_info_df, how="inner")
colombia_df.to_csv('resources/top_chart_features/colombia_df.csv', index=False)

# Costa Rica Top Chart Track Features

In [22]:
#empty lists to store song data
costa_rica_danceability = []
costa_rica_energy = []
costa_rica_key = []
costa_rica_loudness = []
costa_rica_mode = []
costa_rica_speechiness = []
costa_rica_acousticness = []
costa_rica_instrumentalness = []
costa_rica_liveness = []
costa_rica_valence = []
costa_rica_tempo = []
costa_rica_id_num = []
costa_rica_duration_mins = []
costa_rica_time_signature = []

for i in d['costa_rica']["track_id"]:
    costa_rica_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    costa_rica_danceability.append(costa_rica_response["danceability"])
    costa_rica_energy.append(costa_rica_response["energy"])
    costa_rica_key.append(costa_rica_response["key"])
    costa_rica_loudness.append(costa_rica_response["loudness"])
    costa_rica_mode.append(costa_rica_response["mode"])
    costa_rica_speechiness.append(costa_rica_response["speechiness"])
    costa_rica_acousticness.append(costa_rica_response["acousticness"])
    costa_rica_instrumentalness.append(costa_rica_response["instrumentalness"])
    costa_rica_liveness.append(costa_rica_response["liveness"])
    costa_rica_valence.append(costa_rica_response["valence"])
    costa_rica_tempo.append(costa_rica_response["tempo"])
    costa_rica_id_num.append(costa_rica_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    costa_rica_duration_mins.append(round((costa_rica_response["duration_ms"] / 60000),2))
    costa_rica_time_signature.append(costa_rica_response["time_signature"])  

    # time.sleep(1)

costa_rica_dict = {}
#create a dictionary to hold data gathered from the api
costa_rica_dict = {
    "track_id": costa_rica_id_num,
    "danceability": costa_rica_danceability,
    "energy": costa_rica_energy,
    "key": costa_rica_key,
    "loudness": costa_rica_loudness,
    "mode": costa_rica_mode,
    "speechiness": costa_rica_speechiness,
    "acousticness": costa_rica_acousticness,
    "instrumentalness": costa_rica_instrumentalness,
    "liveness": costa_rica_liveness,
    "valence": costa_rica_valence,
    "tempo": costa_rica_tempo,
    "duration_mins": costa_rica_duration_mins,
    "time_signature": costa_rica_time_signature,
}

#convert dictionary to a dataframe
costa_rica_info_df = pd.DataFrame(costa_rica_dict)

costa_rica_tracks = d['costa_rica']
costa_rica_df = costa_rica_tracks.merge(costa_rica_info_df, how="inner")
costa_rica_df['region'] = "Costa Rica"
costa_rica_df.to_csv('resources/top_chart_features/costa_rica_df.csv', index=False)

# Cyprus Top Chart Track Features

In [23]:
#empty lists to store song data
cyprus_danceability = []
cyprus_energy = []
cyprus_key = []
cyprus_loudness = []
cyprus_mode = []
cyprus_speechiness = []
cyprus_acousticness = []
cyprus_instrumentalness = []
cyprus_liveness = []
cyprus_valence = []
cyprus_tempo = []
cyprus_id_num = []
cyprus_duration_mins = []
cyprus_time_signature = []

for i in d['cyprus']["track_id"]:
    cyprus_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    cyprus_danceability.append(cyprus_response["danceability"])
    cyprus_energy.append(cyprus_response["energy"])
    cyprus_key.append(cyprus_response["key"])
    cyprus_loudness.append(cyprus_response["loudness"])
    cyprus_mode.append(cyprus_response["mode"])
    cyprus_speechiness.append(cyprus_response["speechiness"])
    cyprus_acousticness.append(cyprus_response["acousticness"])
    cyprus_instrumentalness.append(cyprus_response["instrumentalness"])
    cyprus_liveness.append(cyprus_response["liveness"])
    cyprus_valence.append(cyprus_response["valence"])
    cyprus_tempo.append(cyprus_response["tempo"])
    cyprus_id_num.append(cyprus_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    cyprus_duration_mins.append(round((cyprus_response["duration_ms"] / 60000),2))
    cyprus_time_signature.append(cyprus_response["time_signature"])  

    # time.sleep(1)

cyprus_dict = {}
#create a dictionary to hold data gathered from the api
cyprus_dict = {
    "track_id": cyprus_id_num,
    "danceability": cyprus_danceability,
    "energy": cyprus_energy,
    "key": cyprus_key,
    "loudness": cyprus_loudness,
    "mode": cyprus_mode,
    "speechiness": cyprus_speechiness,
    "acousticness": cyprus_acousticness,
    "instrumentalness": cyprus_instrumentalness,
    "liveness": cyprus_liveness,
    "valence": cyprus_valence,
    "tempo": cyprus_tempo,
    "duration_mins": cyprus_duration_mins,
    "time_signature": cyprus_time_signature,
}

#convert dictionary to a dataframe
cyprus_info_df = pd.DataFrame(cyprus_dict)
cyprus_info_df.head()


cyprus_tracks = d['cyprus']
cyprus_df = cyprus_tracks.merge(cyprus_info_df, how="inner")
cyprus_df.to_csv('resources/top_chart_features/cyprus_df.csv', index=False)

# Czech Republic Top Chart Track Features

In [24]:
#empty lists to store song data
czech_republic_danceability = []
czech_republic_energy = []
czech_republic_key = []
czech_republic_loudness = []
czech_republic_mode = []
czech_republic_speechiness = []
czech_republic_acousticness = []
czech_republic_instrumentalness = []
czech_republic_liveness = []
czech_republic_valence = []
czech_republic_tempo = []
czech_republic_id_num = []
czech_republic_duration_mins = []
czech_republic_time_signature = []

for i in d['czech_republic']["track_id"]:
    czech_republic_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    czech_republic_danceability.append(czech_republic_response["danceability"])
    czech_republic_energy.append(czech_republic_response["energy"])
    czech_republic_key.append(czech_republic_response["key"])
    czech_republic_loudness.append(czech_republic_response["loudness"])
    czech_republic_mode.append(czech_republic_response["mode"])
    czech_republic_speechiness.append(czech_republic_response["speechiness"])
    czech_republic_acousticness.append(czech_republic_response["acousticness"])
    czech_republic_instrumentalness.append(czech_republic_response["instrumentalness"])
    czech_republic_liveness.append(czech_republic_response["liveness"])
    czech_republic_valence.append(czech_republic_response["valence"])
    czech_republic_tempo.append(czech_republic_response["tempo"])
    czech_republic_id_num.append(czech_republic_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    czech_republic_duration_mins.append(round((czech_republic_response["duration_ms"] / 60000),2))
    czech_republic_time_signature.append(czech_republic_response["time_signature"])  

    # time.sleep(1)

czech_republic_dict = {}
#create a dictionary to hold data gathered from the api
czech_republic_dict = {
    "track_id": czech_republic_id_num,
    "danceability": czech_republic_danceability,
    "energy": czech_republic_energy,
    "key": czech_republic_key,
    "loudness": czech_republic_loudness,
    "mode": czech_republic_mode,
    "speechiness": czech_republic_speechiness,
    "acousticness": czech_republic_acousticness,
    "instrumentalness": czech_republic_instrumentalness,
    "liveness": czech_republic_liveness,
    "valence": czech_republic_valence,
    "tempo": czech_republic_tempo,
    "duration_mins": czech_republic_duration_mins,
    "time_signature": czech_republic_time_signature
}

#convert dictionary to a dataframe
czech_republic_info_df = pd.DataFrame(czech_republic_dict)
czech_republic_info_df.head()


czech_republic_tracks = d['czech_republic']
czech_republic_df = czech_republic_tracks.merge(czech_republic_info_df, how="inner")
czech_republic_df['region'] = "Czech Republic"
czech_republic_df.to_csv('resources/top_chart_features/czech_republic_df.csv', index=False)

# Denmark Top Chart Track Features

In [25]:
#empty lists to store song data
denmark_danceability = []
denmark_energy = []
denmark_key = []
denmark_loudness = []
denmark_mode = []
denmark_speechiness = []
denmark_acousticness = []
denmark_instrumentalness = []
denmark_liveness = []
denmark_valence = []
denmark_tempo = []
denmark_id_num = []
denmark_duration_mins = []
denmark_time_signature = []

for i in d['denmark']["track_id"]:
    denmark_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    denmark_danceability.append(denmark_response["danceability"])
    denmark_energy.append(denmark_response["energy"])
    denmark_key.append(denmark_response["key"])
    denmark_loudness.append(denmark_response["loudness"])
    denmark_mode.append(denmark_response["mode"])
    denmark_speechiness.append(denmark_response["speechiness"])
    denmark_acousticness.append(denmark_response["acousticness"])
    denmark_instrumentalness.append(denmark_response["instrumentalness"])
    denmark_liveness.append(denmark_response["liveness"])
    denmark_valence.append(denmark_response["valence"])
    denmark_tempo.append(denmark_response["tempo"])
    denmark_id_num.append(denmark_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    denmark_duration_mins.append(round((denmark_response["duration_ms"] / 60000),2))
    denmark_time_signature.append(denmark_response["time_signature"])  

    # time.sleep(1)

denmark_dict = {}
#create a dictionary to hold data gathered from the api
denmark_dict = {
    "track_id": denmark_id_num,
    "danceability": denmark_danceability,
    "energy": denmark_energy,
    "key": denmark_key,
    "loudness": denmark_loudness,
    "mode": denmark_mode,
    "speechiness": denmark_speechiness,
    "acousticness": denmark_acousticness,
    "instrumentalness": denmark_instrumentalness,
    "liveness": denmark_liveness,
    "valence": denmark_valence,
    "tempo": denmark_tempo,
    "duration_mins": denmark_duration_mins,
    "time_signature": denmark_time_signature
}

#convert dictionary to a dataframe
denmark_info_df = pd.DataFrame(denmark_dict)

denmark_tracks = d['denmark']
denmark_df = denmark_tracks.merge(denmark_info_df, how="inner")
denmark_df.to_csv('resources/top_chart_features/denmark_df.csv', index=False)

# Dominican Republic Top Chart Track Features

In [26]:
#empty lists to store song data
dominican_republic_danceability = []
dominican_republic_energy = []
dominican_republic_key = []
dominican_republic_loudness = []
dominican_republic_mode = []
dominican_republic_speechiness = []
dominican_republic_acousticness = []
dominican_republic_instrumentalness = []
dominican_republic_liveness = []
dominican_republic_valence = []
dominican_republic_tempo = []
dominican_republic_id_num = []
dominican_republic_duration_mins = []
dominican_republic_time_signature = []

for i in d['dominican_republic']["track_id"]:
    dominican_republic_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    dominican_republic_danceability.append(dominican_republic_response["danceability"])
    dominican_republic_energy.append(dominican_republic_response["energy"])
    dominican_republic_key.append(dominican_republic_response["key"])
    dominican_republic_loudness.append(dominican_republic_response["loudness"])
    dominican_republic_mode.append(dominican_republic_response["mode"])
    dominican_republic_speechiness.append(dominican_republic_response["speechiness"])
    dominican_republic_acousticness.append(dominican_republic_response["acousticness"])
    dominican_republic_instrumentalness.append(dominican_republic_response["instrumentalness"])
    dominican_republic_liveness.append(dominican_republic_response["liveness"])
    dominican_republic_valence.append(dominican_republic_response["valence"])
    dominican_republic_tempo.append(dominican_republic_response["tempo"])
    dominican_republic_id_num.append(dominican_republic_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    dominican_republic_duration_mins.append(round((dominican_republic_response["duration_ms"] / 60000),2))
    dominican_republic_time_signature.append(dominican_republic_response["time_signature"])  

    # time.sleep(1)

dominican_republic_dict = {}
#create a dictionary to hold data gathered from the api
dominican_republic_dict = {
    "track_id": dominican_republic_id_num,
    "danceability": dominican_republic_danceability,
    "energy": dominican_republic_energy,
    "key": dominican_republic_key,
    "loudness": dominican_republic_loudness,
    "mode": dominican_republic_mode,
    "speechiness": dominican_republic_speechiness,
    "acousticness": dominican_republic_acousticness,
    "instrumentalness": dominican_republic_instrumentalness,
    "liveness": dominican_republic_liveness,
    "valence": dominican_republic_valence,
    "tempo": dominican_republic_tempo,
    "duration_mins": dominican_republic_duration_mins,
    "time_signature": dominican_republic_time_signature
}

#convert dictionary to a dataframe
dominican_republic_info_df = pd.DataFrame(dominican_republic_dict)

dominican_republic_tracks = d['dominican_republic']
dominican_republic_df = dominican_republic_tracks.merge(dominican_republic_info_df, how="inner")
dominican_republic_df['region'] = "Dominican Republic"
dominican_republic_df.to_csv('resources/top_chart_features/dominican_republic_df.csv', index=False)

# Ecuador Top Chart Track Features

In [29]:
#empty lists to store song data
ecuador_danceability = []
ecuador_energy = []
ecuador_key = []
ecuador_loudness = []
ecuador_mode = []
ecuador_speechiness = []
ecuador_acousticness = []
ecuador_instrumentalness = []
ecuador_liveness = []
ecuador_valence = []
ecuador_tempo = []
ecuador_id_num = []
ecuador_duration_mins = []
ecuador_time_signature = []

for i in d['ecuador']["track_id"]:
    ecuador_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    ecuador_danceability.append(ecuador_response["danceability"])
    ecuador_energy.append(ecuador_response["energy"])
    ecuador_key.append(ecuador_response["key"])
    ecuador_loudness.append(ecuador_response["loudness"])
    ecuador_mode.append(ecuador_response["mode"])
    ecuador_speechiness.append(ecuador_response["speechiness"])
    ecuador_acousticness.append(ecuador_response["acousticness"])
    ecuador_instrumentalness.append(ecuador_response["instrumentalness"])
    ecuador_liveness.append(ecuador_response["liveness"])
    ecuador_valence.append(ecuador_response["valence"])
    ecuador_tempo.append(ecuador_response["tempo"])
    ecuador_id_num.append(ecuador_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    ecuador_duration_mins.append(round((ecuador_response["duration_ms"] / 60000),2))
    ecuador_time_signature.append(ecuador_response["time_signature"])  

    # time.sleep(1)

ecuador_dict = {}
#create a dictionary to hold data gathered from the api
ecuador_dict = {
    "track_id": ecuador_id_num,
    "danceability": ecuador_danceability,
    "energy": ecuador_energy,
    "key": ecuador_key,
    "loudness": ecuador_loudness,
    "mode": ecuador_mode,
    "speechiness": ecuador_speechiness,
    "acousticness": ecuador_acousticness,
    "instrumentalness": ecuador_instrumentalness,
    "liveness": ecuador_liveness,
    "valence": ecuador_valence,
    "tempo": ecuador_tempo,
    "duration_mins": ecuador_duration_mins,
    "time_signature": ecuador_time_signature
}

#convert dictionary to a dataframe
ecuador_info_df = pd.DataFrame(ecuador_dict)

ecuador_tracks = d['ecuador']
ecuador_df = ecuador_tracks.merge(ecuador_info_df, how="inner")
ecuador_df.to_csv('resources/top_chart_features/ecuador_df.csv', index=False)

# Egypt Top Chart Track Features

In [30]:
#empty lists to store song data
egypt_danceability = []
egypt_energy = []
egypt_key = []
egypt_loudness = []
egypt_mode = []
egypt_speechiness = []
egypt_acousticness = []
egypt_instrumentalness = []
egypt_liveness = []
egypt_valence = []
egypt_tempo = []
egypt_id_num = []
egypt_duration_mins = []
egypt_time_signature = []

for i in d['egypt']["track_id"]:
    egypt_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    egypt_danceability.append(egypt_response["danceability"])
    egypt_energy.append(egypt_response["energy"])
    egypt_key.append(egypt_response["key"])
    egypt_loudness.append(egypt_response["loudness"])
    egypt_mode.append(egypt_response["mode"])
    egypt_speechiness.append(egypt_response["speechiness"])
    egypt_acousticness.append(egypt_response["acousticness"])
    egypt_instrumentalness.append(egypt_response["instrumentalness"])
    egypt_liveness.append(egypt_response["liveness"])
    egypt_valence.append(egypt_response["valence"])
    egypt_tempo.append(egypt_response["tempo"])
    egypt_id_num.append(egypt_response["id"])    
    #duration recorded in api in milliseconds, converting to minutes here
    egypt_duration_mins.append(round((egypt_response["duration_ms"] / 60000),2))
    egypt_time_signature.append(egypt_response["time_signature"])  

    # time.sleep(1)

egypt_dict = {}
#create a dictionary to hold data gathered from the api
egypt_dict = {
    "track_id": egypt_id_num,
    "danceability": egypt_danceability,
    "energy": egypt_energy,
    "key": egypt_key,
    "loudness": egypt_loudness,
    "mode": egypt_mode,
    "speechiness": egypt_speechiness,
    "acousticness": egypt_acousticness,
    "instrumentalness": egypt_instrumentalness,
    "liveness": egypt_liveness,
    "valence": egypt_valence,
    "tempo": egypt_tempo,
    "duration_mins": egypt_duration_mins,
    "time_signature": egypt_time_signature
}

#convert dictionary to a dataframe
egypt_info_df = pd.DataFrame(egypt_dict)
egypt_info_df.head()


egypt_tracks = d['egypt']
egypt_df = egypt_tracks.merge(egypt_info_df, how="inner")
egypt_df.to_csv('resources/top_chart_features/egypt_df.csv', index=False)

# El Salvador Top Chart Track Features

In [31]:
#empty lists to store song data
el_salvador_danceability = []
el_salvador_energy = []
el_salvador_key = []
el_salvador_loudness = []
el_salvador_mode = []
el_salvador_speechiness = []
el_salvador_acousticness = []
el_salvador_instrumentalness = []
el_salvador_liveness = []
el_salvador_valence = []
el_salvador_tempo = []
el_salvador_id_num = []
el_salvador_duration_mins = []
el_salvador_time_signature = []

for i in d['el_salvador']["track_id"]:
    el_salvador_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    el_salvador_danceability.append(el_salvador_response["danceability"])
    el_salvador_energy.append(el_salvador_response["energy"])
    el_salvador_key.append(el_salvador_response["key"])
    el_salvador_loudness.append(el_salvador_response["loudness"])
    el_salvador_mode.append(el_salvador_response["mode"])
    el_salvador_speechiness.append(el_salvador_response["speechiness"])
    el_salvador_acousticness.append(el_salvador_response["acousticness"])
    el_salvador_instrumentalness.append(el_salvador_response["instrumentalness"])
    el_salvador_liveness.append(el_salvador_response["liveness"])
    el_salvador_valence.append(el_salvador_response["valence"])
    el_salvador_tempo.append(el_salvador_response["tempo"])
    el_salvador_id_num.append(el_salvador_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    el_salvador_duration_mins.append(round((el_salvador_response["duration_ms"] / 60000),2))
    el_salvador_time_signature.append(el_salvador_response["time_signature"])  

    # time.sleep(1)

el_salvador_dict = {}
#create a dictionary to hold data gathered from the api
el_salvador_dict = {
    "track_id": el_salvador_id_num,
    "danceability": el_salvador_danceability,
    "energy": el_salvador_energy,
    "key": el_salvador_key,
    "loudness": el_salvador_loudness,
    "mode": el_salvador_mode,
    "speechiness": el_salvador_speechiness,
    "acousticness": el_salvador_acousticness,
    "instrumentalness": el_salvador_instrumentalness,
    "liveness": el_salvador_liveness,
    "valence": el_salvador_valence,
    "tempo": el_salvador_tempo,
    "duration_mins": el_salvador_duration_mins,
    "time_signature": el_salvador_time_signature
}

#convert dictionary to a dataframe
el_salvador_info_df = pd.DataFrame(el_salvador_dict)

el_salvador_tracks = d['el_salvador']
el_salvador_df = el_salvador_tracks.merge(el_salvador_info_df, how="inner")
el_salvador_df['region'] = "El Salvador"
el_salvador_df.to_csv('resources/top_chart_features/el_salvador_df.csv', index=False)

# Estonia Top Chart Track Features

In [32]:
#empty lists to store song data
estonia_danceability = []
estonia_energy = []
estonia_key = []
estonia_loudness = []
estonia_mode = []
estonia_speechiness = []
estonia_acousticness = []
estonia_instrumentalness = []
estonia_liveness = []
estonia_valence = []
estonia_tempo = []
estonia_id_num = []
estonia_duration_mins = []
estonia_time_signature = []

for i in d['estonia']["track_id"]:
    estonia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    estonia_danceability.append(estonia_response["danceability"])
    estonia_energy.append(estonia_response["energy"])
    estonia_key.append(estonia_response["key"])
    estonia_loudness.append(estonia_response["loudness"])
    estonia_mode.append(estonia_response["mode"])
    estonia_speechiness.append(estonia_response["speechiness"])
    estonia_acousticness.append(estonia_response["acousticness"])
    estonia_instrumentalness.append(estonia_response["instrumentalness"])
    estonia_liveness.append(estonia_response["liveness"])
    estonia_valence.append(estonia_response["valence"])
    estonia_tempo.append(estonia_response["tempo"])
    estonia_id_num.append(estonia_response["id"])   
    #duration recorded in api in milliseconds, converting to minutes here
    estonia_duration_mins.append(round((estonia_response["duration_ms"] / 60000),2))
    estonia_time_signature.append(estonia_response["time_signature"])  

    # time.sleep(1)

estonia_dict = {}
#create a dictionary to hold data gathered from the api
estonia_dict = {
    "track_id": estonia_id_num,
    "danceability": estonia_danceability,
    "energy": estonia_energy,
    "key": estonia_key,
    "loudness": estonia_loudness,
    "mode": estonia_mode,
    "speechiness": estonia_speechiness,
    "acousticness": estonia_acousticness,
    "instrumentalness": estonia_instrumentalness,
    "liveness": estonia_liveness,
    "valence": estonia_valence,
    "tempo": estonia_tempo,
    "duration_mins": estonia_duration_mins,
    "time_signature": estonia_time_signature
}

#convert dictionary to a dataframe
estonia_info_df = pd.DataFrame(estonia_dict)

estonia_tracks = d['estonia']
estonia_df = estonia_tracks.merge(estonia_info_df, how="inner")
estonia_df.to_csv('resources/top_chart_features/estonia_df.csv', index=False)

# Finland Top Chart Track Features

In [33]:
#empty lists to store song data
finland_danceability = []
finland_energy = []
finland_key = []
finland_loudness = []
finland_mode = []
finland_speechiness = []
finland_acousticness = []
finland_instrumentalness = []
finland_liveness = []
finland_valence = []
finland_tempo = []
finland_id_num = []
finland_duration_mins = []
finland_time_signature = []

for i in d['finland']["track_id"]:
    finland_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    finland_danceability.append(finland_response["danceability"])
    finland_energy.append(finland_response["energy"])
    finland_key.append(finland_response["key"])
    finland_loudness.append(finland_response["loudness"])
    finland_mode.append(finland_response["mode"])
    finland_speechiness.append(finland_response["speechiness"])
    finland_acousticness.append(finland_response["acousticness"])
    finland_instrumentalness.append(finland_response["instrumentalness"])
    finland_liveness.append(finland_response["liveness"])
    finland_valence.append(finland_response["valence"])
    finland_tempo.append(finland_response["tempo"])
    finland_id_num.append(finland_response["id"])   
    #duration recorded in api in milliseconds, converting to minutes here
    finland_duration_mins.append(round((finland_response["duration_ms"] / 60000),2))
    finland_time_signature.append(finland_response["time_signature"])  

    # time.sleep(1)

finland_dict = {}
#create a dictionary to hold data gathered from the api
finland_dict = {
    "track_id": finland_id_num,
    "danceability": finland_danceability,
    "energy": finland_energy,
    "key": finland_key,
    "loudness": finland_loudness,
    "mode": finland_mode,
    "speechiness": finland_speechiness,
    "acousticness": finland_acousticness,
    "instrumentalness": finland_instrumentalness,
    "liveness": finland_liveness,
    "valence": finland_valence,
    "tempo": finland_tempo,
    "duration_mins": finland_duration_mins,
    "time_signature": finland_time_signature,
}

#convert dictionary to a dataframe
finland_info_df = pd.DataFrame(finland_dict)

finland_tracks = d['finland']
finland_df = finland_tracks.merge(finland_info_df, how="inner")
finland_df.to_csv('resources/top_chart_features/finland_df.csv', index=False)

# France Top Chart Track Features

In [34]:
#empty lists to store song data
france_danceability = []
france_energy = []
france_key = []
france_loudness = []
france_mode = []
france_speechiness = []
france_acousticness = []
france_instrumentalness = []
france_liveness = []
france_valence = []
france_tempo = []
france_id_num = []
france_duration_mins = []
france_time_signature = []

for i in d['france']["track_id"]:
    france_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    france_danceability.append(france_response["danceability"])
    france_energy.append(france_response["energy"])
    france_key.append(france_response["key"])
    france_loudness.append(france_response["loudness"])
    france_mode.append(france_response["mode"])
    france_speechiness.append(france_response["speechiness"])
    france_acousticness.append(france_response["acousticness"])
    france_instrumentalness.append(france_response["instrumentalness"])
    france_liveness.append(france_response["liveness"])
    france_valence.append(france_response["valence"])
    france_tempo.append(france_response["tempo"])
    france_id_num.append(france_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    france_duration_mins.append(round((france_response["duration_ms"] / 60000),2))
    france_time_signature.append(france_response["time_signature"])  

    # time.sleep(1)

france_dict = {}
#create a dictionary to hold data gathered from the api
france_dict = {
    "track_id": france_id_num,
    "danceability": france_danceability,
    "energy": france_energy,
    "key": france_key,
    "loudness": france_loudness,
    "mode": france_mode,
    "speechiness": france_speechiness,
    "acousticness": france_acousticness,
    "instrumentalness": france_instrumentalness,
    "liveness": france_liveness,
    "valence": france_valence,
    "tempo": france_tempo,
    "duration_mins": france_duration_mins,
    "time_signature": france_time_signature
}

#convert dictionary to a dataframe
france_info_df = pd.DataFrame(france_dict)

france_tracks = d['france']
france_df = france_tracks.merge(france_info_df, how="inner")
france_df.to_csv('resources/top_chart_features/france_df.csv', index=False)

# Germany Top Chart Track Features

In [35]:
#empty lists to store song data
germany_danceability = []
germany_energy = []
germany_key = []
germany_loudness = []
germany_mode = []
germany_speechiness = []
germany_acousticness = []
germany_instrumentalness = []
germany_liveness = []
germany_valence = []
germany_tempo = []
germany_id_num = []
germany_duration_mins = []
germany_time_signature = []

for i in d['germany']["track_id"]:
    germany_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    germany_danceability.append(germany_response["danceability"])
    germany_energy.append(germany_response["energy"])
    germany_key.append(germany_response["key"])
    germany_loudness.append(germany_response["loudness"])
    germany_mode.append(germany_response["mode"])
    germany_speechiness.append(germany_response["speechiness"])
    germany_acousticness.append(germany_response["acousticness"])
    germany_instrumentalness.append(germany_response["instrumentalness"])
    germany_liveness.append(germany_response["liveness"])
    germany_valence.append(germany_response["valence"])
    germany_tempo.append(germany_response["tempo"])
    germany_id_num.append(germany_response["id"])    
    #duration recorded in api in milliseconds, converting to minutes here
    germany_duration_mins.append(round((germany_response["duration_ms"] / 60000),2))
    germany_time_signature.append(germany_response["time_signature"])  

    # time.sleep(1)

germany_dict = {}
#create a dictionary to hold data gathered from the api
germany_dict = {
    "track_id": germany_id_num,
    "danceability": germany_danceability,
    "energy": germany_energy,
    "key": germany_key,
    "loudness": germany_loudness,
    "mode": germany_mode,
    "speechiness": germany_speechiness,
    "acousticness": germany_acousticness,
    "instrumentalness": germany_instrumentalness,
    "liveness": germany_liveness,
    "valence": germany_valence,
    "tempo": germany_tempo,
    "duration_mins": germany_duration_mins,
    "time_signature": germany_time_signature
}

#convert dictionary to a dataframe
germany_info_df = pd.DataFrame(germany_dict)

germany_tracks = d['germany']
germany_df = germany_tracks.merge(germany_info_df, how="inner")
germany_df.to_csv('resources/top_chart_features/germany_df.csv', index=False)

# Greece Top Chart Track Features

In [36]:
#empty lists to store song data
greece_danceability = []
greece_energy = []
greece_key = []
greece_loudness = []
greece_mode = []
greece_speechiness = []
greece_acousticness = []
greece_instrumentalness = []
greece_liveness = []
greece_valence = []
greece_tempo = []
greece_id_num = []
greece_duration_mins = []
greece_time_signature = []

for i in d['greece']["track_id"]:
    greece_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    greece_danceability.append(greece_response["danceability"])
    greece_energy.append(greece_response["energy"])
    greece_key.append(greece_response["key"])
    greece_loudness.append(greece_response["loudness"])
    greece_mode.append(greece_response["mode"])
    greece_speechiness.append(greece_response["speechiness"])
    greece_acousticness.append(greece_response["acousticness"])
    greece_instrumentalness.append(greece_response["instrumentalness"])
    greece_liveness.append(greece_response["liveness"])
    greece_valence.append(greece_response["valence"])
    greece_tempo.append(greece_response["tempo"])
    greece_id_num.append(greece_response["id"]) 
    #duration recorded in api in milliseconds, converting to minutes here
    greece_duration_mins.append(round((greece_response["duration_ms"] / 60000),2))
    greece_time_signature.append(greece_response["time_signature"])  

    # time.sleep(1)

greece_dict = {}
#create a dictionary to hold data gathered from the api
greece_dict = {
    "track_id": greece_id_num,
    "danceability": greece_danceability,
    "energy": greece_energy,
    "key": greece_key,
    "loudness": greece_loudness,
    "mode": greece_mode,
    "speechiness": greece_speechiness,
    "acousticness": greece_acousticness,
    "instrumentalness": greece_instrumentalness,
    "liveness": greece_liveness,
    "valence": greece_valence,
    "tempo": greece_tempo,
    "duration_mins": greece_duration_mins,
    "time_signature": greece_time_signature
}

#convert dictionary to a dataframe
greece_info_df = pd.DataFrame(greece_dict)

greece_tracks = d['greece']
greece_df = greece_tracks.merge(greece_info_df, how="inner")
greece_df.to_csv('resources/top_chart_features/greece_df.csv', index=False)

# Guatemala Top Chart Track Features

In [37]:
#empty lists to store song data
guatemala_danceability = []
guatemala_energy = []
guatemala_key = []
guatemala_loudness = []
guatemala_mode = []
guatemala_speechiness = []
guatemala_acousticness = []
guatemala_instrumentalness = []
guatemala_liveness = []
guatemala_valence = []
guatemala_tempo = []
guatemala_id_num = []
guatemala_duration_mins = []
guatemala_time_signature = []

for i in d['guatemala']["track_id"]:
    guatemala_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    guatemala_danceability.append(guatemala_response["danceability"])
    guatemala_energy.append(guatemala_response["energy"])
    guatemala_key.append(guatemala_response["key"])
    guatemala_loudness.append(guatemala_response["loudness"])
    guatemala_mode.append(guatemala_response["mode"])
    guatemala_speechiness.append(guatemala_response["speechiness"])
    guatemala_acousticness.append(guatemala_response["acousticness"])
    guatemala_instrumentalness.append(guatemala_response["instrumentalness"])
    guatemala_liveness.append(guatemala_response["liveness"])
    guatemala_valence.append(guatemala_response["valence"])
    guatemala_tempo.append(guatemala_response["tempo"])
    guatemala_id_num.append(guatemala_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    guatemala_duration_mins.append(round((guatemala_response["duration_ms"] / 60000),2))
    guatemala_time_signature.append(guatemala_response["time_signature"])  

    # time.sleep(1)

guatemala_dict = {}
#create a dictionary to hold data gathered from the api
guatemala_dict = {
    "track_id": guatemala_id_num,
    "danceability": guatemala_danceability,
    "energy": guatemala_energy,
    "key": guatemala_key,
    "loudness": guatemala_loudness,
    "mode": guatemala_mode,
    "speechiness": guatemala_speechiness,
    "acousticness": guatemala_acousticness,
    "instrumentalness": guatemala_instrumentalness,
    "liveness": guatemala_liveness,
    "valence": guatemala_valence,
    "tempo": guatemala_tempo,
    "duration_mins": guatemala_duration_mins,
    "time_signature": guatemala_time_signature
}

#convert dictionary to a dataframe
guatemala_info_df = pd.DataFrame(guatemala_dict)

guatemala_tracks = d['guatemala']
guatemala_df = guatemala_tracks.merge(guatemala_info_df, how="inner")
guatemala_df.to_csv('resources/top_chart_features/guatemala_df.csv', index=False)

# Honduras Top Chart Track Features

In [38]:
#empty lists to store song data
honduras_danceability = []
honduras_energy = []
honduras_key = []
honduras_loudness = []
honduras_mode = []
honduras_speechiness = []
honduras_acousticness = []
honduras_instrumentalness = []
honduras_liveness = []
honduras_valence = []
honduras_tempo = []
honduras_id_num = []
honduras_duration_mins = []
honduras_time_signature = []

for i in d['honduras']["track_id"]:
    honduras_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    honduras_danceability.append(honduras_response["danceability"])
    honduras_energy.append(honduras_response["energy"])
    honduras_key.append(honduras_response["key"])
    honduras_loudness.append(honduras_response["loudness"])
    honduras_mode.append(honduras_response["mode"])
    honduras_speechiness.append(honduras_response["speechiness"])
    honduras_acousticness.append(honduras_response["acousticness"])
    honduras_instrumentalness.append(honduras_response["instrumentalness"])
    honduras_liveness.append(honduras_response["liveness"])
    honduras_valence.append(honduras_response["valence"])
    honduras_tempo.append(honduras_response["tempo"])
    honduras_id_num.append(honduras_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    honduras_duration_mins.append(round((honduras_response["duration_ms"] / 60000),2))
    honduras_time_signature.append(honduras_response["time_signature"])  

    # time.sleep(1)

honduras_dict = {}
#create a dictionary to hold data gathered from the api
honduras_dict = {
    "track_id": honduras_id_num,
    "danceability": honduras_danceability,
    "energy": honduras_energy,
    "key": honduras_key,
    "loudness": honduras_loudness,
    "mode": honduras_mode,
    "speechiness": honduras_speechiness,
    "acousticness": honduras_acousticness,
    "instrumentalness": honduras_instrumentalness,
    "liveness": honduras_liveness,
    "valence": honduras_valence,
    "tempo": honduras_tempo,
    "duration_mins": honduras_duration_mins,
    "time_signature": honduras_time_signature
}

#convert dictionary to a dataframe
honduras_info_df = pd.DataFrame(honduras_dict)

honduras_tracks = d['honduras']
honduras_df = honduras_tracks.merge(honduras_info_df, how="inner")
honduras_df.to_csv('resources/top_chart_features/honduras_df.csv', index=False)

# Hong Kong Top Chart Track Features

In [39]:
#empty lists to store song data
hong_kong_danceability = []
hong_kong_energy = []
hong_kong_key = []
hong_kong_loudness = []
hong_kong_mode = []
hong_kong_speechiness = []
hong_kong_acousticness = []
hong_kong_instrumentalness = []
hong_kong_liveness = []
hong_kong_valence = []
hong_kong_tempo = []
hong_kong_id_num = []
hong_kong_duration_mins = []
hong_kong_time_signature = []

for i in d['hong_kong']["track_id"]:
    hong_kong_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    hong_kong_danceability.append(hong_kong_response["danceability"])
    hong_kong_energy.append(hong_kong_response["energy"])
    hong_kong_key.append(hong_kong_response["key"])
    hong_kong_loudness.append(hong_kong_response["loudness"])
    hong_kong_mode.append(hong_kong_response["mode"])
    hong_kong_speechiness.append(hong_kong_response["speechiness"])
    hong_kong_acousticness.append(hong_kong_response["acousticness"])
    hong_kong_instrumentalness.append(hong_kong_response["instrumentalness"])
    hong_kong_liveness.append(hong_kong_response["liveness"])
    hong_kong_valence.append(hong_kong_response["valence"])
    hong_kong_tempo.append(hong_kong_response["tempo"])
    hong_kong_id_num.append(hong_kong_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    hong_kong_duration_mins.append(round((hong_kong_response["duration_ms"] / 60000),2))
    hong_kong_time_signature.append(hong_kong_response["time_signature"])  

    # time.sleep(1)

hong_kong_dict = {}
#create a dictionary to hold data gathered from the api
hong_kong_dict = {
    "track_id": hong_kong_id_num,
    "danceability": hong_kong_danceability,
    "energy": hong_kong_energy,
    "key": hong_kong_key,
    "loudness": hong_kong_loudness,
    "mode": hong_kong_mode,
    "speechiness": hong_kong_speechiness,
    "acousticness": hong_kong_acousticness,
    "instrumentalness": hong_kong_instrumentalness,
    "liveness": hong_kong_liveness,
    "valence": hong_kong_valence,
    "tempo": hong_kong_tempo,
    "duration_mins": hong_kong_duration_mins,
    "time_signature": hong_kong_time_signature
}

#convert dictionary to a dataframe
hong_kong_info_df = pd.DataFrame(hong_kong_dict)

hong_kong_tracks = d['hong_kong']
hong_kong_df = hong_kong_tracks.merge(hong_kong_info_df, how="inner")
hong_kong_df['region'] = "Hong Kong"
hong_kong_df.to_csv('resources/top_chart_features/hong_kong_df.csv', index=False)

# Hungary Top Chart Track Features

In [40]:
#empty lists to store song data
hungary_danceability = []
hungary_energy = []
hungary_key = []
hungary_loudness = []
hungary_mode = []
hungary_speechiness = []
hungary_acousticness = []
hungary_instrumentalness = []
hungary_liveness = []
hungary_valence = []
hungary_tempo = []
hungary_id_num = []
hungary_duration_mins = []
hungary_time_signature = []

for i in d['hungary']["track_id"]:
    hungary_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    hungary_danceability.append(hungary_response["danceability"])
    hungary_energy.append(hungary_response["energy"])
    hungary_key.append(hungary_response["key"])
    hungary_loudness.append(hungary_response["loudness"])
    hungary_mode.append(hungary_response["mode"])
    hungary_speechiness.append(hungary_response["speechiness"])
    hungary_acousticness.append(hungary_response["acousticness"])
    hungary_instrumentalness.append(hungary_response["instrumentalness"])
    hungary_liveness.append(hungary_response["liveness"])
    hungary_valence.append(hungary_response["valence"])
    hungary_tempo.append(hungary_response["tempo"])
    hungary_id_num.append(hungary_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    hungary_duration_mins.append(round((hungary_response["duration_ms"] / 60000),2))
    hungary_time_signature.append(hungary_response["time_signature"])  

    # time.sleep(1)

hungary_dict = {}
#create a dictionary to hold data gathered from the api
hungary_dict = {
    "track_id": hungary_id_num,
    "danceability": hungary_danceability,
    "energy": hungary_energy,
    "key": hungary_key,
    "loudness": hungary_loudness,
    "mode": hungary_mode,
    "speechiness": hungary_speechiness,
    "acousticness": hungary_acousticness,
    "instrumentalness": hungary_instrumentalness,
    "liveness": hungary_liveness,
    "valence": hungary_valence,
    "tempo": hungary_tempo,
    "duration_mins": hungary_duration_mins,
    "time_signature": hungary_time_signature
}

#convert dictionary to a dataframe
hungary_info_df = pd.DataFrame(hungary_dict)

hungary_tracks = d['hungary']
hungary_df = hungary_tracks.merge(hungary_info_df, how="inner")
hungary_df.to_csv('resources/top_chart_features/hungary_df.csv', index=False)

# Iceland Top Chart Track Features

In [45]:
# keeps throwing errors

# #empty lists to store song data
# iceland_danceability = []
# iceland_energy = []
# iceland_key = []
# iceland_loudness = []
# iceland_mode = []
# iceland_speechiness = []
# iceland_acousticness = []
# iceland_instrumentalness = []
# iceland_liveness = []
# iceland_valence = []
# iceland_tempo = []
# iceland_id_num = []
# iceland_duration_mins = []
# iceland_time_signature = []

# for i in d['iceland']["track_id"]:
#     # time.sleep(1)
#     iceland_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
#     #fill in track info for each audio feature / empty list
#     iceland_danceability.append(iceland_response["danceability"])
#     iceland_energy.append(iceland_response["energy"])
#     iceland_key.append(iceland_response["key"])
#     iceland_loudness.append(iceland_response["loudness"])
#     iceland_mode.append(iceland_response["mode"])
#     iceland_speechiness.append(iceland_response["speechiness"])
#     iceland_acousticness.append(iceland_response["acousticness"])
#     iceland_instrumentalness.append(iceland_response["instrumentalness"])
#     iceland_liveness.append(iceland_response["liveness"])
#     iceland_valence.append(iceland_response["valence"])
#     iceland_tempo.append(iceland_response["tempo"])
#     iceland_id_num.append(iceland_response["id"])  
#     #duration recorded in api in milliseconds, converting to minutes here
#     iceland_duration_mins.append(round((iceland_response["duration_ms"] / 60000),2))
#     iceland_time_signature.append(iceland_response["time_signature"])  

#     # time.sleep(1)

# iceland_dict = {}
# #create a dictionary to hold data gathered from the api
# iceland_dict = {
#     "track_id": iceland_id_num,
#     "danceability": iceland_danceability,
#     "energy": iceland_energy,
#     "key": iceland_key,
#     "loudness": iceland_loudness,
#     "mode": iceland_mode,
#     "speechiness": iceland_speechiness,
#     "acousticness": iceland_acousticness,
#     "instrumentalness": iceland_instrumentalness,
#     "liveness": iceland_liveness,
#     "valence": iceland_valence,
#     "tempo": iceland_tempo,
#     "duration_mins": iceland_duration_mins,
#     "time_signature": iceland_time_signature
# }

# #convert dictionary to a dataframe
# iceland_info_df = pd.DataFrame(iceland_dict)

# iceland_tracks = d['iceland']
# iceland_df = iceland_tracks.merge(iceland_info_df, how="inner")
# iceland_df.to_csv('resources/top_chart_features/iceland_df.csv', index=False)

# India Top Chart Track Features

In [47]:
# keeps throwing errors

# #empty lists to store song data
# india_danceability = []
# india_energy = []
# india_key = []
# india_loudness = []
# india_mode = []
# india_speechiness = []
# india_acousticness = []
# india_instrumentalness = []
# india_liveness = []
# india_valence = []
# india_tempo = []
# india_id_num = []
# india_duration_mins = []
# india_time_signature = []

# for i in d['india']["track_id"]:
#     india_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
#     #fill in track info for each audio feature / empty list
#     india_danceability.append(india_response["danceability"])
#     india_energy.append(india_response["energy"])
#     india_key.append(india_response["key"])
#     india_loudness.append(india_response["loudness"])
#     india_mode.append(india_response["mode"])
#     india_speechiness.append(india_response["speechiness"])
#     india_acousticness.append(india_response["acousticness"])
#     india_instrumentalness.append(india_response["instrumentalness"])
#     india_liveness.append(india_response["liveness"])
#     india_valence.append(india_response["valence"])
#     india_tempo.append(india_response["tempo"])
#     india_id_num.append(india_response["id"])  
#     #duration recorded in api in milliseconds, converting to minutes here
#     india_duration_mins.append(round((india_response["duration_ms"] / 60000),2))
#     india_time_signature.append(india_response["time_signature"])  

#     # time.sleep(1)

# india_dict = {}
# #create a dictionary to hold data gathered from the api
# india_dict = {
#     "track_id": india_id_num,
#     "danceability": india_danceability,
#     "energy": india_energy,
#     "key": india_key,
#     "loudness": india_loudness,
#     "mode": india_mode,
#     "speechiness": india_speechiness,
#     "acousticness": india_acousticness,
#     "instrumentalness": india_instrumentalness,
#     "liveness": india_liveness,
#     "valence": india_valence,
#     "tempo": india_tempo,
#     "duration_mins": india_duration_mins,
#     "time_signature": india_time_signature
# }

# #convert dictionary to a dataframe
# india_info_df = pd.DataFrame(india_dict)

# india_tracks = d['india']
# india_df = india_tracks.merge(india_info_df, how="inner")
# india_df.to_csv('resources/top_chart_features/india_df.csv', index=False)

# Idonesia Top Chart Track Features

In [48]:
#empty lists to store song data
indonesia_danceability = []
indonesia_energy = []
indonesia_key = []
indonesia_loudness = []
indonesia_mode = []
indonesia_speechiness = []
indonesia_acousticness = []
indonesia_instrumentalness = []
indonesia_liveness = []
indonesia_valence = []
indonesia_tempo = []
indonesia_id_num = []
indonesia_duration_mins = []
indonesia_time_signature = []

for i in d['indonesia']["track_id"]:
    indonesia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    indonesia_danceability.append(indonesia_response["danceability"])
    indonesia_energy.append(indonesia_response["energy"])
    indonesia_key.append(indonesia_response["key"])
    indonesia_loudness.append(indonesia_response["loudness"])
    indonesia_mode.append(indonesia_response["mode"])
    indonesia_speechiness.append(indonesia_response["speechiness"])
    indonesia_acousticness.append(indonesia_response["acousticness"])
    indonesia_instrumentalness.append(indonesia_response["instrumentalness"])
    indonesia_liveness.append(indonesia_response["liveness"])
    indonesia_valence.append(indonesia_response["valence"])
    indonesia_tempo.append(indonesia_response["tempo"])
    indonesia_id_num.append(indonesia_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    indonesia_duration_mins.append(round((indonesia_response["duration_ms"] / 60000),2))
    indonesia_time_signature.append(indonesia_response["time_signature"])  

    # time.sleep(1)

indonesia_dict = {}
#create a dictionary to hold data gathered from the api
indonesia_dict = {
    "track_id": indonesia_id_num,
    "danceability": indonesia_danceability,
    "energy": indonesia_energy,
    "key": indonesia_key,
    "loudness": indonesia_loudness,
    "mode": indonesia_mode,
    "speechiness": indonesia_speechiness,
    "acousticness": indonesia_acousticness,
    "instrumentalness": indonesia_instrumentalness,
    "liveness": indonesia_liveness,
    "valence": indonesia_valence,
    "tempo": indonesia_tempo,
    "duration_mins": indonesia_duration_mins,
    "time_signature": indonesia_time_signature
}

#convert dictionary to a dataframe
indonesia_info_df = pd.DataFrame(indonesia_dict)

indonesia_tracks = d['indonesia']
indonesia_df = indonesia_tracks.merge(indonesia_info_df, how="inner")
indonesia_df.to_csv('resources/top_chart_features/indonesia_df.csv', index=False)

# Ireland Top Chart Track Features

In [49]:
#empty lists to store song data
ireland_danceability = []
ireland_energy = []
ireland_key = []
ireland_loudness = []
ireland_mode = []
ireland_speechiness = []
ireland_acousticness = []
ireland_instrumentalness = []
ireland_liveness = []
ireland_valence = []
ireland_tempo = []
ireland_id_num = []
ireland_duration_mins = []
ireland_time_signature = []

for i in d['ireland']["track_id"]:
    ireland_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    ireland_danceability.append(ireland_response["danceability"])
    ireland_energy.append(ireland_response["energy"])
    ireland_key.append(ireland_response["key"])
    ireland_loudness.append(ireland_response["loudness"])
    ireland_mode.append(ireland_response["mode"])
    ireland_speechiness.append(ireland_response["speechiness"])
    ireland_acousticness.append(ireland_response["acousticness"])
    ireland_instrumentalness.append(ireland_response["instrumentalness"])
    ireland_liveness.append(ireland_response["liveness"])
    ireland_valence.append(ireland_response["valence"])
    ireland_tempo.append(ireland_response["tempo"])
    ireland_id_num.append(ireland_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    ireland_duration_mins.append(round((ireland_response["duration_ms"] / 60000),2))
    ireland_time_signature.append(ireland_response["time_signature"])  

    # time.sleep(1)

ireland_dict = {}
#create a dictionary to hold data gathered from the api
ireland_dict = {
    "track_id": ireland_id_num,
    "danceability": ireland_danceability,
    "energy": ireland_energy,
    "key": ireland_key,
    "loudness": ireland_loudness,
    "mode": ireland_mode,
    "speechiness": ireland_speechiness,
    "acousticness": ireland_acousticness,
    "instrumentalness": ireland_instrumentalness,
    "liveness": ireland_liveness,
    "valence": ireland_valence,
    "tempo": ireland_tempo,
    "duration_mins": ireland_duration_mins,
    "time_signature": ireland_time_signature
}

#convert dictionary to a dataframe
ireland_info_df = pd.DataFrame(ireland_dict)

ireland_tracks = d['ireland']
ireland_df = ireland_tracks.merge(ireland_info_df, how="inner")
ireland_df.to_csv('resources/top_chart_features/ireland_df.csv', index=False)

# Israel Top Chart Track Features

In [50]:
#empty lists to store song data
israel_danceability = []
israel_energy = []
israel_key = []
israel_loudness = []
israel_mode = []
israel_speechiness = []
israel_acousticness = []
israel_instrumentalness = []
israel_liveness = []
israel_valence = []
israel_tempo = []
israel_id_num = []
israel_duration_mins = []
israel_time_signature = []

for i in d['israel']["track_id"]:
    israel_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    israel_danceability.append(israel_response["danceability"])
    israel_energy.append(israel_response["energy"])
    israel_key.append(israel_response["key"])
    israel_loudness.append(israel_response["loudness"])
    israel_mode.append(israel_response["mode"])
    israel_speechiness.append(israel_response["speechiness"])
    israel_acousticness.append(israel_response["acousticness"])
    israel_instrumentalness.append(israel_response["instrumentalness"])
    israel_liveness.append(israel_response["liveness"])
    israel_valence.append(israel_response["valence"])
    israel_tempo.append(israel_response["tempo"])
    israel_id_num.append(israel_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    israel_duration_mins.append(round((israel_response["duration_ms"] / 60000),2))
    israel_time_signature.append(israel_response["time_signature"])  

    # time.sleep(1)

israel_dict = {}
#create a dictionary to hold data gathered from the api
israel_dict = {
    "track_id": israel_id_num,
    "danceability": israel_danceability,
    "energy": israel_energy,
    "key": israel_key,
    "loudness": israel_loudness,
    "mode": israel_mode,
    "speechiness": israel_speechiness,
    "acousticness": israel_acousticness,
    "instrumentalness": israel_instrumentalness,
    "liveness": israel_liveness,
    "valence": israel_valence,
    "tempo": israel_tempo,
    "duration_mins": israel_duration_mins,
    "time_signature": israel_time_signature
}

#convert dictionary to a dataframe
israel_info_df = pd.DataFrame(israel_dict)

israel_tracks = d['israel']
israel_df = israel_tracks.merge(israel_info_df, how="inner")
israel_df.to_csv('resources/top_chart_features/israel_df.csv', index=False)

# Italy Top Chart Track Features

In [51]:
#empty lists to store song data
italy_danceability = []
italy_energy = []
italy_key = []
italy_loudness = []
italy_mode = []
italy_speechiness = []
italy_acousticness = []
italy_instrumentalness = []
italy_liveness = []
italy_valence = []
italy_tempo = []
italy_id_num = []
italy_duration_mins = []
italy_time_signature = []

for i in d['italy']["track_id"]:
    italy_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    italy_danceability.append(italy_response["danceability"])
    italy_energy.append(italy_response["energy"])
    italy_key.append(italy_response["key"])
    italy_loudness.append(italy_response["loudness"])
    italy_mode.append(italy_response["mode"])
    italy_speechiness.append(italy_response["speechiness"])
    italy_acousticness.append(italy_response["acousticness"])
    italy_instrumentalness.append(italy_response["instrumentalness"])
    italy_liveness.append(italy_response["liveness"])
    italy_valence.append(italy_response["valence"])
    italy_tempo.append(italy_response["tempo"])
    italy_id_num.append(italy_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    italy_duration_mins.append(round((italy_response["duration_ms"] / 60000),2))
    italy_time_signature.append(italy_response["time_signature"])  

    # time.sleep(1)

italy_dict = {}
#create a dictionary to hold data gathered from the api
italy_dict = {
    "track_id": italy_id_num,
    "danceability": italy_danceability,
    "energy": italy_energy,
    "key": italy_key,
    "loudness": italy_loudness,
    "mode": italy_mode,
    "speechiness": italy_speechiness,
    "acousticness": italy_acousticness,
    "instrumentalness": italy_instrumentalness,
    "liveness": italy_liveness,
    "valence": italy_valence,
    "tempo": italy_tempo,
    "duration_mins": italy_duration_mins,
    "time_signature": italy_time_signature
}

#convert dictionary to a dataframe
italy_info_df = pd.DataFrame(italy_dict)

italy_tracks = d['italy']
italy_df = italy_tracks.merge(italy_info_df, how="inner")
italy_df.to_csv('resources/top_chart_features/italy_df.csv', index=False)

# Japan Top Chart Track Features

In [52]:
#empty lists to store song data
japan_danceability = []
japan_energy = []
japan_key = []
japan_loudness = []
japan_mode = []
japan_speechiness = []
japan_acousticness = []
japan_instrumentalness = []
japan_liveness = []
japan_valence = []
japan_tempo = []
japan_id_num = []
japan_duration_mins = []
japan_time_signature = []

for i in d['japan']["track_id"]:
    japan_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    japan_danceability.append(japan_response["danceability"])
    japan_energy.append(japan_response["energy"])
    japan_key.append(japan_response["key"])
    japan_loudness.append(japan_response["loudness"])
    japan_mode.append(japan_response["mode"])
    japan_speechiness.append(japan_response["speechiness"])
    japan_acousticness.append(japan_response["acousticness"])
    japan_instrumentalness.append(japan_response["instrumentalness"])
    japan_liveness.append(japan_response["liveness"])
    japan_valence.append(japan_response["valence"])
    japan_tempo.append(japan_response["tempo"])
    japan_id_num.append(japan_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    japan_duration_mins.append(round((japan_response["duration_ms"] / 60000),2))
    japan_time_signature.append(japan_response["time_signature"])  

    # time.sleep(1)

japan_dict = {}
#create a dictionary to hold data gathered from the api
japan_dict = {
    "track_id": japan_id_num,
    "danceability": japan_danceability,
    "energy": japan_energy,
    "key": japan_key,
    "loudness": japan_loudness,
    "mode": japan_mode,
    "speechiness": japan_speechiness,
    "acousticness": japan_acousticness,
    "instrumentalness": japan_instrumentalness,
    "liveness": japan_liveness,
    "valence": japan_valence,
    "tempo": japan_tempo,
    "duration_mins": japan_duration_mins,
    "time_signature": japan_time_signature
}

#convert dictionary to a dataframe
japan_info_df = pd.DataFrame(japan_dict)

japan_tracks = d['japan']
japan_df = japan_tracks.merge(japan_info_df, how="inner")
japan_df.to_csv('resources/top_chart_features/japan_df.csv', index=False)

# Kazakhstan Top Chart Track Features

In [53]:
#empty lists to store song data
kazakhstan_danceability = []
kazakhstan_energy = []
kazakhstan_key = []
kazakhstan_loudness = []
kazakhstan_mode = []
kazakhstan_speechiness = []
kazakhstan_acousticness = []
kazakhstan_instrumentalness = []
kazakhstan_liveness = []
kazakhstan_valence = []
kazakhstan_tempo = []
kazakhstan_id_num = []
kazakhstan_duration_mins = []
kazakhstan_time_signature = []

for i in d['kazakhstan']["track_id"]:
    kazakhstan_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    kazakhstan_danceability.append(kazakhstan_response["danceability"])
    kazakhstan_energy.append(kazakhstan_response["energy"])
    kazakhstan_key.append(kazakhstan_response["key"])
    kazakhstan_loudness.append(kazakhstan_response["loudness"])
    kazakhstan_mode.append(kazakhstan_response["mode"])
    kazakhstan_speechiness.append(kazakhstan_response["speechiness"])
    kazakhstan_acousticness.append(kazakhstan_response["acousticness"])
    kazakhstan_instrumentalness.append(kazakhstan_response["instrumentalness"])
    kazakhstan_liveness.append(kazakhstan_response["liveness"])
    kazakhstan_valence.append(kazakhstan_response["valence"])
    kazakhstan_tempo.append(kazakhstan_response["tempo"])
    kazakhstan_id_num.append(kazakhstan_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    kazakhstan_duration_mins.append(round((kazakhstan_response["duration_ms"] / 60000),2))
    kazakhstan_time_signature.append(kazakhstan_response["time_signature"])  

    # time.sleep(1)

kazakhstan_dict = {}
#create a dictionary to hold data gathered from the api
kazakhstan_dict = {
    "track_id": kazakhstan_id_num,
    "danceability": kazakhstan_danceability,
    "energy": kazakhstan_energy,
    "key": kazakhstan_key,
    "loudness": kazakhstan_loudness,
    "mode": kazakhstan_mode,
    "speechiness": kazakhstan_speechiness,
    "acousticness": kazakhstan_acousticness,
    "instrumentalness": kazakhstan_instrumentalness,
    "liveness": kazakhstan_liveness,
    "valence": kazakhstan_valence,
    "tempo": kazakhstan_tempo,
    "duration_mins": kazakhstan_duration_mins,
    "time_signature": kazakhstan_time_signature
}

#convert dictionary to a dataframe
kazakhstan_info_df = pd.DataFrame(kazakhstan_dict)

kazakhstan_tracks = d['kazakhstan']
kazakhstan_df = kazakhstan_tracks.merge(kazakhstan_info_df, how="inner")
kazakhstan_df.to_csv('resources/top_chart_features/kazakhstan_df.csv', index=False)

# Latvia Top Chart Track Features

In [54]:
#empty lists to store song data
latvia_danceability = []
latvia_energy = []
latvia_key = []
latvia_loudness = []
latvia_mode = []
latvia_speechiness = []
latvia_acousticness = []
latvia_instrumentalness = []
latvia_liveness = []
latvia_valence = []
latvia_tempo = []
latvia_id_num = []
latvia_duration_mins = []
latvia_time_signature = []

for i in d['latvia']["track_id"]:
    latvia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    latvia_danceability.append(latvia_response["danceability"])
    latvia_energy.append(latvia_response["energy"])
    latvia_key.append(latvia_response["key"])
    latvia_loudness.append(latvia_response["loudness"])
    latvia_mode.append(latvia_response["mode"])
    latvia_speechiness.append(latvia_response["speechiness"])
    latvia_acousticness.append(latvia_response["acousticness"])
    latvia_instrumentalness.append(latvia_response["instrumentalness"])
    latvia_liveness.append(latvia_response["liveness"])
    latvia_valence.append(latvia_response["valence"])
    latvia_tempo.append(latvia_response["tempo"])
    latvia_id_num.append(latvia_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    latvia_duration_mins.append(round((latvia_response["duration_ms"] / 60000),2))
    latvia_time_signature.append(latvia_response["time_signature"])  

    # time.sleep(1)

latvia_dict = {}
#create a dictionary to hold data gathered from the api
latvia_dict = {
    "track_id": latvia_id_num,
    "danceability": latvia_danceability,
    "energy": latvia_energy,
    "key": latvia_key,
    "loudness": latvia_loudness,
    "mode": latvia_mode,
    "speechiness": latvia_speechiness,
    "acousticness": latvia_acousticness,
    "instrumentalness": latvia_instrumentalness,
    "liveness": latvia_liveness,
    "valence": latvia_valence,
    "tempo": latvia_tempo,
    "duration_mins": latvia_duration_mins,
    "time_signature": latvia_time_signature
}

#convert dictionary to a dataframe
latvia_info_df = pd.DataFrame(latvia_dict)

latvia_tracks = d['latvia']
latvia_df = latvia_tracks.merge(latvia_info_df, how="inner")
latvia_df.to_csv('resources/top_chart_features/latvia_df.csv', index=False)

# Lithuania Top Chart Track Features

In [55]:
#empty lists to store song data
lithuania_danceability = []
lithuania_energy = []
lithuania_key = []
lithuania_loudness = []
lithuania_mode = []
lithuania_speechiness = []
lithuania_acousticness = []
lithuania_instrumentalness = []
lithuania_liveness = []
lithuania_valence = []
lithuania_tempo = []
lithuania_id_num = []
lithuania_duration_mins = []
lithuania_time_signature = []

for i in d['lithuania']["track_id"]:
    lithuania_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    lithuania_danceability.append(lithuania_response["danceability"])
    lithuania_energy.append(lithuania_response["energy"])
    lithuania_key.append(lithuania_response["key"])
    lithuania_loudness.append(lithuania_response["loudness"])
    lithuania_mode.append(lithuania_response["mode"])
    lithuania_speechiness.append(lithuania_response["speechiness"])
    lithuania_acousticness.append(lithuania_response["acousticness"])
    lithuania_instrumentalness.append(lithuania_response["instrumentalness"])
    lithuania_liveness.append(lithuania_response["liveness"])
    lithuania_valence.append(lithuania_response["valence"])
    lithuania_tempo.append(lithuania_response["tempo"])
    lithuania_id_num.append(lithuania_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    lithuania_duration_mins.append(round((lithuania_response["duration_ms"] / 60000),2))
    lithuania_time_signature.append(lithuania_response["time_signature"])  

    # time.sleep(1)

lithuania_dict = {}
#create a dictionary to hold data gathered from the api
lithuania_dict = {
    "track_id": lithuania_id_num,
    "danceability": lithuania_danceability,
    "energy": lithuania_energy,
    "key": lithuania_key,
    "loudness": lithuania_loudness,
    "mode": lithuania_mode,
    "speechiness": lithuania_speechiness,
    "acousticness": lithuania_acousticness,
    "instrumentalness": lithuania_instrumentalness,
    "liveness": lithuania_liveness,
    "valence": lithuania_valence,
    "tempo": lithuania_tempo,
    "duration_mins": lithuania_duration_mins,
    "time_signature": lithuania_time_signature
}

#convert dictionary to a dataframe
lithuania_info_df = pd.DataFrame(lithuania_dict)

lithuania_tracks = d['lithuania']
lithuania_df = lithuania_tracks.merge(lithuania_info_df, how="inner")
lithuania_df.to_csv('resources/top_chart_features/lithuania_df.csv', index=False)

# Luxembourg Top Chart Track Features

In [56]:
#empty lists to store song data
luxembourg_danceability = []
luxembourg_energy = []
luxembourg_key = []
luxembourg_loudness = []
luxembourg_mode = []
luxembourg_speechiness = []
luxembourg_acousticness = []
luxembourg_instrumentalness = []
luxembourg_liveness = []
luxembourg_valence = []
luxembourg_tempo = []
luxembourg_id_num = []
luxembourg_duration_mins = []
luxembourg_time_signature = []

for i in d['luxembourg']["track_id"]:
    luxembourg_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    luxembourg_danceability.append(luxembourg_response["danceability"])
    luxembourg_energy.append(luxembourg_response["energy"])
    luxembourg_key.append(luxembourg_response["key"])
    luxembourg_loudness.append(luxembourg_response["loudness"])
    luxembourg_mode.append(luxembourg_response["mode"])
    luxembourg_speechiness.append(luxembourg_response["speechiness"])
    luxembourg_acousticness.append(luxembourg_response["acousticness"])
    luxembourg_instrumentalness.append(luxembourg_response["instrumentalness"])
    luxembourg_liveness.append(luxembourg_response["liveness"])
    luxembourg_valence.append(luxembourg_response["valence"])
    luxembourg_tempo.append(luxembourg_response["tempo"])
    luxembourg_id_num.append(luxembourg_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    luxembourg_duration_mins.append(round((luxembourg_response["duration_ms"] / 60000),2))
    luxembourg_time_signature.append(luxembourg_response["time_signature"])  

    # time.sleep(1)

luxembourg_dict = {}
#create a dictionary to hold data gathered from the api
luxembourg_dict = {
    "track_id": luxembourg_id_num,
    "danceability": luxembourg_danceability,
    "energy": luxembourg_energy,
    "key": luxembourg_key,
    "loudness": luxembourg_loudness,
    "mode": luxembourg_mode,
    "speechiness": luxembourg_speechiness,
    "acousticness": luxembourg_acousticness,
    "instrumentalness": luxembourg_instrumentalness,
    "liveness": luxembourg_liveness,
    "valence": luxembourg_valence,
    "tempo": luxembourg_tempo,
    "duration_mins": luxembourg_duration_mins,
    "time_signature": luxembourg_time_signature
}

#convert dictionary to a dataframe
luxembourg_info_df = pd.DataFrame(luxembourg_dict)

luxembourg_tracks = d['luxembourg']
luxembourg_df = luxembourg_tracks.merge(luxembourg_info_df, how="inner")
luxembourg_df.to_csv('resources/top_chart_features/luxembourg_df.csv', index=False)

# Malaysia Top Chart Track Features

In [57]:
#empty lists to store song data
malaysia_danceability = []
malaysia_energy = []
malaysia_key = []
malaysia_loudness = []
malaysia_mode = []
malaysia_speechiness = []
malaysia_acousticness = []
malaysia_instrumentalness = []
malaysia_liveness = []
malaysia_valence = []
malaysia_tempo = []
malaysia_id_num = []
malaysia_duration_mins = []
malaysia_time_signature = []

for i in d['malaysia']["track_id"]:
    malaysia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    malaysia_danceability.append(malaysia_response["danceability"])
    malaysia_energy.append(malaysia_response["energy"])
    malaysia_key.append(malaysia_response["key"])
    malaysia_loudness.append(malaysia_response["loudness"])
    malaysia_mode.append(malaysia_response["mode"])
    malaysia_speechiness.append(malaysia_response["speechiness"])
    malaysia_acousticness.append(malaysia_response["acousticness"])
    malaysia_instrumentalness.append(malaysia_response["instrumentalness"])
    malaysia_liveness.append(malaysia_response["liveness"])
    malaysia_valence.append(malaysia_response["valence"])
    malaysia_tempo.append(malaysia_response["tempo"])
    malaysia_id_num.append(malaysia_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    malaysia_duration_mins.append(round((malaysia_response["duration_ms"] / 60000),2))
    malaysia_time_signature.append(malaysia_response["time_signature"])  

    # time.sleep(1)

malaysia_dict = {}
#create a dictionary to hold data gathered from the api
malaysia_dict = {
    "track_id": malaysia_id_num,
    "danceability": malaysia_danceability,
    "energy": malaysia_energy,
    "key": malaysia_key,
    "loudness": malaysia_loudness,
    "mode": malaysia_mode,
    "speechiness": malaysia_speechiness,
    "acousticness": malaysia_acousticness,
    "instrumentalness": malaysia_instrumentalness,
    "liveness": malaysia_liveness,
    "valence": malaysia_valence,
    "tempo": malaysia_tempo,
    "duration_mins": malaysia_duration_mins,
    "time_signature": malaysia_time_signature
}

#convert dictionary to a dataframe
malaysia_info_df = pd.DataFrame(malaysia_dict)

malaysia_tracks = d['malaysia']
malaysia_df = malaysia_tracks.merge(malaysia_info_df, how="inner")
malaysia_df.to_csv('resources/top_chart_features/malaysia_df.csv', index=False)

# Mexico Top Chart Track Features

In [58]:
#empty lists to store song data
mexico_danceability = []
mexico_energy = []
mexico_key = []
mexico_loudness = []
mexico_mode = []
mexico_speechiness = []
mexico_acousticness = []
mexico_instrumentalness = []
mexico_liveness = []
mexico_valence = []
mexico_tempo = []
mexico_id_num = []
mexico_duration_mins = []
mexico_time_signature = []

for i in d['mexico']["track_id"]:
    mexico_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    mexico_danceability.append(mexico_response["danceability"])
    mexico_energy.append(mexico_response["energy"])
    mexico_key.append(mexico_response["key"])
    mexico_loudness.append(mexico_response["loudness"])
    mexico_mode.append(mexico_response["mode"])
    mexico_speechiness.append(mexico_response["speechiness"])
    mexico_acousticness.append(mexico_response["acousticness"])
    mexico_instrumentalness.append(mexico_response["instrumentalness"])
    mexico_liveness.append(mexico_response["liveness"])
    mexico_valence.append(mexico_response["valence"])
    mexico_tempo.append(mexico_response["tempo"])
    mexico_id_num.append(mexico_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    mexico_duration_mins.append(round((mexico_response["duration_ms"] / 60000),2))
    mexico_time_signature.append(mexico_response["time_signature"])  

    # time.sleep(1)

mexico_dict = {}
#create a dictionary to hold data gathered from the api
mexico_dict = {
    "track_id": mexico_id_num,
    "danceability": mexico_danceability,
    "energy": mexico_energy,
    "key": mexico_key,
    "loudness": mexico_loudness,
    "mode": mexico_mode,
    "speechiness": mexico_speechiness,
    "acousticness": mexico_acousticness,
    "instrumentalness": mexico_instrumentalness,
    "liveness": mexico_liveness,
    "valence": mexico_valence,
    "tempo": mexico_tempo,
    "duration_mins": mexico_duration_mins,
    "time_signature": mexico_time_signature
}

#convert dictionary to a dataframe
mexico_info_df = pd.DataFrame(mexico_dict)

mexico_tracks = d['mexico']
mexico_df = mexico_tracks.merge(mexico_info_df, how="inner")
mexico_df.to_csv('resources/top_chart_features/mexico_df.csv', index=False)

# Morocco Top Chart Track Features

In [60]:
# keeps throwing errors

# #empty lists to store song data
# morocco_danceability = []
# morocco_energy = []
# morocco_key = []
# morocco_loudness = []
# morocco_mode = []
# morocco_speechiness = []
# morocco_acousticness = []
# morocco_instrumentalness = []
# morocco_liveness = []
# morocco_valence = []
# morocco_tempo = []
# morocco_id_num = []
# morocco_duration_mins = []
# morocco_time_signature = []

# for i in d['morocco']["track_id"]:
#     morocco_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
#     #fill in track info for each audio feature / empty list
#     morocco_danceability.append(morocco_response["danceability"])
#     morocco_energy.append(morocco_response["energy"])
#     morocco_key.append(morocco_response["key"])
#     morocco_loudness.append(morocco_response["loudness"])
#     morocco_mode.append(morocco_response["mode"])
#     morocco_speechiness.append(morocco_response["speechiness"])
#     morocco_acousticness.append(morocco_response["acousticness"])
#     morocco_instrumentalness.append(morocco_response["instrumentalness"])
#     morocco_liveness.append(morocco_response["liveness"])
#     morocco_valence.append(morocco_response["valence"])
#     morocco_tempo.append(morocco_response["tempo"])
#     morocco_id_num.append(morocco_response["id"])  
#     #duration recorded in api in milliseconds, converting to minutes here
#     morocco_duration_mins.append(round((morocco_response["duration_ms"] / 60000),2))
#     morocco_time_signature.append(morocco_response["time_signature"])  

#     # time.sleep(1)

# morocco_dict = {}
# #create a dictionary to hold data gathered from the api
# morocco_dict = {
#     "track_id": morocco_id_num,
#     "danceability": morocco_danceability,
#     "energy": morocco_energy,
#     "key": morocco_key,
#     "loudness": morocco_loudness,
#     "mode": morocco_mode,
#     "speechiness": morocco_speechiness,
#     "acousticness": morocco_acousticness,
#     "instrumentalness": morocco_instrumentalness,
#     "liveness": morocco_liveness,
#     "valence": morocco_valence,
#     "tempo": morocco_tempo,
#     "duration_mins": morocco_duration_mins,
#     "time_signature": morocco_time_signature
# }

# #convert dictionary to a dataframe
# morocco_info_df = pd.DataFrame(morocco_dict)

# morocco_tracks = d['morocco']
# morocco_df = morocco_tracks.merge(morocco_info_df, how="inner")
# morocco_df.to_csv('resources/top_chart_features/morocco_df.csv', index=False)

# Netherlands Top Chart Track Features

In [63]:
#empty lists to store song data
netherlands_danceability = []
netherlands_energy = []
netherlands_key = []
netherlands_loudness = []
netherlands_mode = []
netherlands_speechiness = []
netherlands_acousticness = []
netherlands_instrumentalness = []
netherlands_liveness = []
netherlands_valence = []
netherlands_tempo = []
netherlands_id_num = []
netherlands_duration_mins = []
netherlands_time_signature = []

for i in d['netherlands']["track_id"]:
    netherlands_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    netherlands_danceability.append(netherlands_response["danceability"])
    netherlands_energy.append(netherlands_response["energy"])
    netherlands_key.append(netherlands_response["key"])
    netherlands_loudness.append(netherlands_response["loudness"])
    netherlands_mode.append(netherlands_response["mode"])
    netherlands_speechiness.append(netherlands_response["speechiness"])
    netherlands_acousticness.append(netherlands_response["acousticness"])
    netherlands_instrumentalness.append(netherlands_response["instrumentalness"])
    netherlands_liveness.append(netherlands_response["liveness"])
    netherlands_valence.append(netherlands_response["valence"])
    netherlands_tempo.append(netherlands_response["tempo"])
    netherlands_id_num.append(netherlands_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    netherlands_duration_mins.append(round((netherlands_response["duration_ms"] / 60000),2))
    netherlands_time_signature.append(netherlands_response["time_signature"])  

    # time.sleep(1)

netherlands_dict = {}
#create a dictionary to hold data gathered from the api
netherlands_dict = {
    "track_id": netherlands_id_num,
    "danceability": netherlands_danceability,
    "energy": netherlands_energy,
    "key": netherlands_key,
    "loudness": netherlands_loudness,
    "mode": netherlands_mode,
    "speechiness": netherlands_speechiness,
    "acousticness": netherlands_acousticness,
    "instrumentalness": netherlands_instrumentalness,
    "liveness": netherlands_liveness,
    "valence": netherlands_valence,
    "tempo": netherlands_tempo,
    "duration_mins": netherlands_duration_mins,
    "time_signature": netherlands_time_signature
}

#convert dictionary to a dataframe
netherlands_info_df = pd.DataFrame(netherlands_dict)

netherlands_tracks = d['netherlands']
netherlands_df = netherlands_tracks.merge(netherlands_info_df, how="inner")
netherlands_df.to_csv('resources/top_chart_features/netherlands_df.csv', index=False)

# New Zealand Top Chart Track Features

In [64]:
#empty lists to store song data
new_zealand_danceability = []
new_zealand_energy = []
new_zealand_key = []
new_zealand_loudness = []
new_zealand_mode = []
new_zealand_speechiness = []
new_zealand_acousticness = []
new_zealand_instrumentalness = []
new_zealand_liveness = []
new_zealand_valence = []
new_zealand_tempo = []
new_zealand_id_num = []
new_zealand_duration_mins = []
new_zealand_time_signature = []

for i in d['new_zealand']["track_id"]:
    new_zealand_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    new_zealand_danceability.append(new_zealand_response["danceability"])
    new_zealand_energy.append(new_zealand_response["energy"])
    new_zealand_key.append(new_zealand_response["key"])
    new_zealand_loudness.append(new_zealand_response["loudness"])
    new_zealand_mode.append(new_zealand_response["mode"])
    new_zealand_speechiness.append(new_zealand_response["speechiness"])
    new_zealand_acousticness.append(new_zealand_response["acousticness"])
    new_zealand_instrumentalness.append(new_zealand_response["instrumentalness"])
    new_zealand_liveness.append(new_zealand_response["liveness"])
    new_zealand_valence.append(new_zealand_response["valence"])
    new_zealand_tempo.append(new_zealand_response["tempo"])
    new_zealand_id_num.append(new_zealand_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    new_zealand_duration_mins.append(round((new_zealand_response["duration_ms"] / 60000),2))
    new_zealand_time_signature.append(new_zealand_response["time_signature"])  

    # time.sleep(1)

new_zealand_dict = {}
#create a dictionary to hold data gathered from the api
new_zealand_dict = {
    "track_id": new_zealand_id_num,
    "danceability": new_zealand_danceability,
    "energy": new_zealand_energy,
    "key": new_zealand_key,
    "loudness": new_zealand_loudness,
    "mode": new_zealand_mode,
    "speechiness": new_zealand_speechiness,
    "acousticness": new_zealand_acousticness,
    "instrumentalness": new_zealand_instrumentalness,
    "liveness": new_zealand_liveness,
    "valence": new_zealand_valence,
    "tempo": new_zealand_tempo,
    "duration_mins": new_zealand_duration_mins,
    "time_signature": new_zealand_time_signature
}

#convert dictionary to a dataframe
new_zealand_info_df = pd.DataFrame(new_zealand_dict)

new_zealand_tracks = d['new_zealand']
new_zealand_df = new_zealand_tracks.merge(new_zealand_info_df, how="inner")
new_zealand_df['region'] = "New Zealand"
new_zealand_df.to_csv('resources/top_chart_features/new_zealand_df.csv', index=False)

# Nicaragua Top Chart Track Features

In [65]:
#empty lists to store song data
nicaragua_danceability = []
nicaragua_energy = []
nicaragua_key = []
nicaragua_loudness = []
nicaragua_mode = []
nicaragua_speechiness = []
nicaragua_acousticness = []
nicaragua_instrumentalness = []
nicaragua_liveness = []
nicaragua_valence = []
nicaragua_tempo = []
nicaragua_id_num = []
nicaragua_duration_mins = []
nicaragua_time_signature = []

for i in d['nicaragua']["track_id"]:
    nicaragua_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    nicaragua_danceability.append(nicaragua_response["danceability"])
    nicaragua_energy.append(nicaragua_response["energy"])
    nicaragua_key.append(nicaragua_response["key"])
    nicaragua_loudness.append(nicaragua_response["loudness"])
    nicaragua_mode.append(nicaragua_response["mode"])
    nicaragua_speechiness.append(nicaragua_response["speechiness"])
    nicaragua_acousticness.append(nicaragua_response["acousticness"])
    nicaragua_instrumentalness.append(nicaragua_response["instrumentalness"])
    nicaragua_liveness.append(nicaragua_response["liveness"])
    nicaragua_valence.append(nicaragua_response["valence"])
    nicaragua_tempo.append(nicaragua_response["tempo"])
    nicaragua_id_num.append(nicaragua_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    nicaragua_duration_mins.append(round((nicaragua_response["duration_ms"] / 60000),2))
    nicaragua_time_signature.append(nicaragua_response["time_signature"])  

    # time.sleep(1)

nicaragua_dict = {}
#create a dictionary to hold data gathered from the api
nicaragua_dict = {
    "track_id": nicaragua_id_num,
    "danceability": nicaragua_danceability,
    "energy": nicaragua_energy,
    "key": nicaragua_key,
    "loudness": nicaragua_loudness,
    "mode": nicaragua_mode,
    "speechiness": nicaragua_speechiness,
    "acousticness": nicaragua_acousticness,
    "instrumentalness": nicaragua_instrumentalness,
    "liveness": nicaragua_liveness,
    "valence": nicaragua_valence,
    "tempo": nicaragua_tempo,
    "duration_mins": nicaragua_duration_mins,
    "time_signature": nicaragua_time_signature
}

#convert dictionary to a dataframe
nicaragua_info_df = pd.DataFrame(nicaragua_dict)

nicaragua_tracks = d['nicaragua']
nicaragua_df = nicaragua_tracks.merge(nicaragua_info_df, how="inner")
nicaragua_df.to_csv('resources/top_chart_features/nicaragua_df.csv', index=False)

# Nigeria Top Chart Track Features

In [66]:
#empty lists to store song data
nigeria_danceability = []
nigeria_energy = []
nigeria_key = []
nigeria_loudness = []
nigeria_mode = []
nigeria_speechiness = []
nigeria_acousticness = []
nigeria_instrumentalness = []
nigeria_liveness = []
nigeria_valence = []
nigeria_tempo = []
nigeria_id_num = []
nigeria_duration_mins = []
nigeria_time_signature = []

for i in d['nigeria']["track_id"]:
    nigeria_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    nigeria_danceability.append(nigeria_response["danceability"])
    nigeria_energy.append(nigeria_response["energy"])
    nigeria_key.append(nigeria_response["key"])
    nigeria_loudness.append(nigeria_response["loudness"])
    nigeria_mode.append(nigeria_response["mode"])
    nigeria_speechiness.append(nigeria_response["speechiness"])
    nigeria_acousticness.append(nigeria_response["acousticness"])
    nigeria_instrumentalness.append(nigeria_response["instrumentalness"])
    nigeria_liveness.append(nigeria_response["liveness"])
    nigeria_valence.append(nigeria_response["valence"])
    nigeria_tempo.append(nigeria_response["tempo"])
    nigeria_id_num.append(nigeria_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    nigeria_duration_mins.append(round((nigeria_response["duration_ms"] / 60000),2))
    nigeria_time_signature.append(nigeria_response["time_signature"])  

    # time.sleep(1)

nigeria_dict = {}
#create a dictionary to hold data gathered from the api
nigeria_dict = {
    "track_id": nigeria_id_num,
    "danceability": nigeria_danceability,
    "energy": nigeria_energy,
    "key": nigeria_key,
    "loudness": nigeria_loudness,
    "mode": nigeria_mode,
    "speechiness": nigeria_speechiness,
    "acousticness": nigeria_acousticness,
    "instrumentalness": nigeria_instrumentalness,
    "liveness": nigeria_liveness,
    "valence": nigeria_valence,
    "tempo": nigeria_tempo,
    "duration_mins": nigeria_duration_mins,
    "time_signature": nigeria_time_signature
}

#convert dictionary to a dataframe
nigeria_info_df = pd.DataFrame(nigeria_dict)

nigeria_tracks = d['nigeria']
nigeria_df = nigeria_tracks.merge(nigeria_info_df, how="inner")
nigeria_df.to_csv('resources/top_chart_features/nigeria_df.csv', index=False)

# Norway Top Chart Track Features

In [67]:
#empty lists to store song data
norway_danceability = []
norway_energy = []
norway_key = []
norway_loudness = []
norway_mode = []
norway_speechiness = []
norway_acousticness = []
norway_instrumentalness = []
norway_liveness = []
norway_valence = []
norway_tempo = []
norway_id_num = []
norway_duration_mins = []
norway_time_signature = []

for i in d['norway']["track_id"]:
    norway_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    norway_danceability.append(norway_response["danceability"])
    norway_energy.append(norway_response["energy"])
    norway_key.append(norway_response["key"])
    norway_loudness.append(norway_response["loudness"])
    norway_mode.append(norway_response["mode"])
    norway_speechiness.append(norway_response["speechiness"])
    norway_acousticness.append(norway_response["acousticness"])
    norway_instrumentalness.append(norway_response["instrumentalness"])
    norway_liveness.append(norway_response["liveness"])
    norway_valence.append(norway_response["valence"])
    norway_tempo.append(norway_response["tempo"])
    norway_id_num.append(norway_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    norway_duration_mins.append(round((norway_response["duration_ms"] / 60000),2))
    norway_time_signature.append(norway_response["time_signature"])  

    # time.sleep(1)

norway_dict = {}
#create a dictionary to hold data gathered from the api
norway_dict = {
    "track_id": norway_id_num,
    "danceability": norway_danceability,
    "energy": norway_energy,
    "key": norway_key,
    "loudness": norway_loudness,
    "mode": norway_mode,
    "speechiness": norway_speechiness,
    "acousticness": norway_acousticness,
    "instrumentalness": norway_instrumentalness,
    "liveness": norway_liveness,
    "valence": norway_valence,
    "tempo": norway_tempo,
    "duration_mins": norway_duration_mins,
    "time_signature": norway_time_signature
}

#convert dictionary to a dataframe
norway_info_df = pd.DataFrame(norway_dict)

norway_tracks = d['norway']
norway_df = norway_tracks.merge(norway_info_df, how="inner")
norway_df.to_csv('resources/top_chart_features/norway_df.csv', index=False)

# Pakistan Top Chart Track Features

In [68]:
#empty lists to store song data
pakistan_danceability = []
pakistan_energy = []
pakistan_key = []
pakistan_loudness = []
pakistan_mode = []
pakistan_speechiness = []
pakistan_acousticness = []
pakistan_instrumentalness = []
pakistan_liveness = []
pakistan_valence = []
pakistan_tempo = []
pakistan_id_num = []
pakistan_duration_mins = []
pakistan_time_signature = []

for i in d['pakistan']["track_id"]:
    pakistan_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    pakistan_danceability.append(pakistan_response["danceability"])
    pakistan_energy.append(pakistan_response["energy"])
    pakistan_key.append(pakistan_response["key"])
    pakistan_loudness.append(pakistan_response["loudness"])
    pakistan_mode.append(pakistan_response["mode"])
    pakistan_speechiness.append(pakistan_response["speechiness"])
    pakistan_acousticness.append(pakistan_response["acousticness"])
    pakistan_instrumentalness.append(pakistan_response["instrumentalness"])
    pakistan_liveness.append(pakistan_response["liveness"])
    pakistan_valence.append(pakistan_response["valence"])
    pakistan_tempo.append(pakistan_response["tempo"])
    pakistan_id_num.append(pakistan_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    pakistan_duration_mins.append(round((pakistan_response["duration_ms"] / 60000),2))
    pakistan_time_signature.append(pakistan_response["time_signature"])  

    # time.sleep(1)

pakistan_dict = {}
#create a dictionary to hold data gathered from the api
pakistan_dict = {
    "track_id": pakistan_id_num,
    "danceability": pakistan_danceability,
    "energy": pakistan_energy,
    "key": pakistan_key,
    "loudness": pakistan_loudness,
    "mode": pakistan_mode,
    "speechiness": pakistan_speechiness,
    "acousticness": pakistan_acousticness,
    "instrumentalness": pakistan_instrumentalness,
    "liveness": pakistan_liveness,
    "valence": pakistan_valence,
    "tempo": pakistan_tempo,
    "duration_mins": pakistan_duration_mins,
    "time_signature": pakistan_time_signature
}

#convert dictionary to a dataframe
pakistan_info_df = pd.DataFrame(pakistan_dict)

pakistan_tracks = d['pakistan']
pakistan_df = pakistan_tracks.merge(pakistan_info_df, how="inner")
pakistan_df.to_csv('resources/top_chart_features/pakistan_df.csv', index=False)

# Panama Top Chart Track Features

In [69]:
#empty lists to store song data
panama_danceability = []
panama_energy = []
panama_key = []
panama_loudness = []
panama_mode = []
panama_speechiness = []
panama_acousticness = []
panama_instrumentalness = []
panama_liveness = []
panama_valence = []
panama_tempo = []
panama_id_num = []
panama_duration_mins = []
panama_time_signature = []

for i in d['panama']["track_id"]:
    panama_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    panama_danceability.append(panama_response["danceability"])
    panama_energy.append(panama_response["energy"])
    panama_key.append(panama_response["key"])
    panama_loudness.append(panama_response["loudness"])
    panama_mode.append(panama_response["mode"])
    panama_speechiness.append(panama_response["speechiness"])
    panama_acousticness.append(panama_response["acousticness"])
    panama_instrumentalness.append(panama_response["instrumentalness"])
    panama_liveness.append(panama_response["liveness"])
    panama_valence.append(panama_response["valence"])
    panama_tempo.append(panama_response["tempo"])
    panama_id_num.append(panama_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    panama_duration_mins.append(round((panama_response["duration_ms"] / 60000),2))
    panama_time_signature.append(panama_response["time_signature"])  

    # time.sleep(1)

panama_dict = {}
#create a dictionary to hold data gathered from the api
panama_dict = {
    "track_id": panama_id_num,
    "danceability": panama_danceability,
    "energy": panama_energy,
    "key": panama_key,
    "loudness": panama_loudness,
    "mode": panama_mode,
    "speechiness": panama_speechiness,
    "acousticness": panama_acousticness,
    "instrumentalness": panama_instrumentalness,
    "liveness": panama_liveness,
    "valence": panama_valence,
    "tempo": panama_tempo,
    "duration_mins": panama_duration_mins,
    "time_signature": panama_time_signature
}

#convert dictionary to a dataframe
panama_info_df = pd.DataFrame(panama_dict)

panama_tracks = d['panama']
panama_df = panama_tracks.merge(panama_info_df, how="inner")
panama_df.to_csv('resources/top_chart_features/panama_df.csv', index=False)

# Paraguay Top Chart Track Features

In [70]:
#empty lists to store song data
paraguay_danceability = []
paraguay_energy = []
paraguay_key = []
paraguay_loudness = []
paraguay_mode = []
paraguay_speechiness = []
paraguay_acousticness = []
paraguay_instrumentalness = []
paraguay_liveness = []
paraguay_valence = []
paraguay_tempo = []
paraguay_id_num = []
paraguay_duration_mins = []
paraguay_time_signature = []

for i in d['paraguay']["track_id"]:
    paraguay_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    paraguay_danceability.append(paraguay_response["danceability"])
    paraguay_energy.append(paraguay_response["energy"])
    paraguay_key.append(paraguay_response["key"])
    paraguay_loudness.append(paraguay_response["loudness"])
    paraguay_mode.append(paraguay_response["mode"])
    paraguay_speechiness.append(paraguay_response["speechiness"])
    paraguay_acousticness.append(paraguay_response["acousticness"])
    paraguay_instrumentalness.append(paraguay_response["instrumentalness"])
    paraguay_liveness.append(paraguay_response["liveness"])
    paraguay_valence.append(paraguay_response["valence"])
    paraguay_tempo.append(paraguay_response["tempo"])
    paraguay_id_num.append(paraguay_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    paraguay_duration_mins.append(round((paraguay_response["duration_ms"] / 60000),2))
    paraguay_time_signature.append(paraguay_response["time_signature"])  

    # time.sleep(1)

paraguay_dict = {}
#create a dictionary to hold data gathered from the api
paraguay_dict = {
    "track_id": paraguay_id_num,
    "danceability": paraguay_danceability,
    "energy": paraguay_energy,
    "key": paraguay_key,
    "loudness": paraguay_loudness,
    "mode": paraguay_mode,
    "speechiness": paraguay_speechiness,
    "acousticness": paraguay_acousticness,
    "instrumentalness": paraguay_instrumentalness,
    "liveness": paraguay_liveness,
    "valence": paraguay_valence,
    "tempo": paraguay_tempo,
    "duration_mins": paraguay_duration_mins,
    "time_signature": paraguay_time_signature
}

#convert dictionary to a dataframe
paraguay_info_df = pd.DataFrame(paraguay_dict)

paraguay_tracks = d['paraguay']
paraguay_df = paraguay_tracks.merge(paraguay_info_df, how="inner")
paraguay_df.to_csv('resources/top_chart_features/paraguay_df.csv', index=False)

# Peru Top Chart Track Features

In [71]:
#empty lists to store song data
peru_danceability = []
peru_energy = []
peru_key = []
peru_loudness = []
peru_mode = []
peru_speechiness = []
peru_acousticness = []
peru_instrumentalness = []
peru_liveness = []
peru_valence = []
peru_tempo = []
peru_id_num = []
peru_duration_mins = []
peru_time_signature = []

for i in d['peru']["track_id"]:
    peru_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    peru_danceability.append(peru_response["danceability"])
    peru_energy.append(peru_response["energy"])
    peru_key.append(peru_response["key"])
    peru_loudness.append(peru_response["loudness"])
    peru_mode.append(peru_response["mode"])
    peru_speechiness.append(peru_response["speechiness"])
    peru_acousticness.append(peru_response["acousticness"])
    peru_instrumentalness.append(peru_response["instrumentalness"])
    peru_liveness.append(peru_response["liveness"])
    peru_valence.append(peru_response["valence"])
    peru_tempo.append(peru_response["tempo"])
    peru_id_num.append(peru_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    peru_duration_mins.append(round((peru_response["duration_ms"] / 60000),2))
    peru_time_signature.append(peru_response["time_signature"])  

    # time.sleep(1)

peru_dict = {}
#create a dictionary to hold data gathered from the api
peru_dict = {
    "track_id": peru_id_num,
    "danceability": peru_danceability,
    "energy": peru_energy,
    "key": peru_key,
    "loudness": peru_loudness,
    "mode": peru_mode,
    "speechiness": peru_speechiness,
    "acousticness": peru_acousticness,
    "instrumentalness": peru_instrumentalness,
    "liveness": peru_liveness,
    "valence": peru_valence,
    "tempo": peru_tempo,
    "duration_mins": peru_duration_mins,
    "time_signature": peru_time_signature
}

#convert dictionary to a dataframe
peru_info_df = pd.DataFrame(peru_dict)

peru_tracks = d['peru']
peru_df = peru_info_df.merge(peru_tracks, how="inner")
peru_df.to_csv('resources/top_chart_features/peru_df.csv', index=False)

# Philippines Top Chart Track Features

In [72]:
#empty lists to store song data
philippines_danceability = []
philippines_energy = []
philippines_key = []
philippines_loudness = []
philippines_mode = []
philippines_speechiness = []
philippines_acousticness = []
philippines_instrumentalness = []
philippines_liveness = []
philippines_valence = []
philippines_tempo = []
philippines_id_num = []
philippines_duration_mins = []
philippines_time_signature = []

for i in d['philippines']["track_id"]:
    philippines_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    philippines_danceability.append(philippines_response["danceability"])
    philippines_energy.append(philippines_response["energy"])
    philippines_key.append(philippines_response["key"])
    philippines_loudness.append(philippines_response["loudness"])
    philippines_mode.append(philippines_response["mode"])
    philippines_speechiness.append(philippines_response["speechiness"])
    philippines_acousticness.append(philippines_response["acousticness"])
    philippines_instrumentalness.append(philippines_response["instrumentalness"])
    philippines_liveness.append(philippines_response["liveness"])
    philippines_valence.append(philippines_response["valence"])
    philippines_tempo.append(philippines_response["tempo"])
    philippines_id_num.append(philippines_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    philippines_duration_mins.append(round((philippines_response["duration_ms"] / 60000),2))
    philippines_time_signature.append(philippines_response["time_signature"])  

    # time.sleep(1)

philippines_dict = {}
#create a dictionary to hold data gathered from the api
philippines_dict = {
    "track_id": philippines_id_num,
    "danceability": philippines_danceability,
    "energy": philippines_energy,
    "key": philippines_key,
    "loudness": philippines_loudness,
    "mode": philippines_mode,
    "speechiness": philippines_speechiness,
    "acousticness": philippines_acousticness,
    "instrumentalness": philippines_instrumentalness,
    "liveness": philippines_liveness,
    "valence": philippines_valence,
    "tempo": philippines_tempo,
    "duration_mins": philippines_duration_mins,
    "time_signature": philippines_time_signature
}

#convert dictionary to a dataframe
philippines_info_df = pd.DataFrame(philippines_dict)

philippines_tracks = d['philippines']
philippines_df = philippines_tracks.merge(philippines_info_df, how="inner")
philippines_df.to_csv('resources/top_chart_features/philippines_df.csv', index=False)

# Poland Top Chart Track Features

In [73]:
#empty lists to store song data
poland_danceability = []
poland_energy = []
poland_key = []
poland_loudness = []
poland_mode = []
poland_speechiness = []
poland_acousticness = []
poland_instrumentalness = []
poland_liveness = []
poland_valence = []
poland_tempo = []
poland_id_num = []
poland_duration_mins = []
poland_time_signature = []

for i in d['poland']["track_id"]:
    poland_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    poland_danceability.append(poland_response["danceability"])
    poland_energy.append(poland_response["energy"])
    poland_key.append(poland_response["key"])
    poland_loudness.append(poland_response["loudness"])
    poland_mode.append(poland_response["mode"])
    poland_speechiness.append(poland_response["speechiness"])
    poland_acousticness.append(poland_response["acousticness"])
    poland_instrumentalness.append(poland_response["instrumentalness"])
    poland_liveness.append(poland_response["liveness"])
    poland_valence.append(poland_response["valence"])
    poland_tempo.append(poland_response["tempo"])
    poland_id_num.append(poland_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    poland_duration_mins.append(round((poland_response["duration_ms"] / 60000),2))
    poland_time_signature.append(poland_response["time_signature"])  

    # time.sleep(1)

poland_dict = {}
#create a dictionary to hold data gathered from the api
poland_dict = {
    "track_id": poland_id_num,
    "danceability": poland_danceability,
    "energy": poland_energy,
    "key": poland_key,
    "loudness": poland_loudness,
    "mode": poland_mode,
    "speechiness": poland_speechiness,
    "acousticness": poland_acousticness,
    "instrumentalness": poland_instrumentalness,
    "liveness": poland_liveness,
    "valence": poland_valence,
    "tempo": poland_tempo,
    "duration_mins": poland_duration_mins,
    "time_signature": poland_time_signature
}

#convert dictionary to a dataframe
poland_info_df = pd.DataFrame(poland_dict)

poland_tracks = d['poland']
poland_df = poland_tracks.merge(poland_info_df, how="inner")
poland_df.to_csv('resources/top_chart_features/poland_df.csv', index=False)

# Portugal Top Chart Track Features

In [74]:
#empty lists to store song data
portugal_danceability = []
portugal_energy = []
portugal_key = []
portugal_loudness = []
portugal_mode = []
portugal_speechiness = []
portugal_acousticness = []
portugal_instrumentalness = []
portugal_liveness = []
portugal_valence = []
portugal_tempo = []
portugal_id_num = []
portugal_duration_mins = []
portugal_time_signature = []

for i in d['portugal']["track_id"]:
    portugal_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    portugal_danceability.append(portugal_response["danceability"])
    portugal_energy.append(portugal_response["energy"])
    portugal_key.append(portugal_response["key"])
    portugal_loudness.append(portugal_response["loudness"])
    portugal_mode.append(portugal_response["mode"])
    portugal_speechiness.append(portugal_response["speechiness"])
    portugal_acousticness.append(portugal_response["acousticness"])
    portugal_instrumentalness.append(portugal_response["instrumentalness"])
    portugal_liveness.append(portugal_response["liveness"])
    portugal_valence.append(portugal_response["valence"])
    portugal_tempo.append(portugal_response["tempo"])
    portugal_id_num.append(portugal_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    portugal_duration_mins.append(round((portugal_response["duration_ms"] / 60000),2))
    portugal_time_signature.append(portugal_response["time_signature"])  

    # time.sleep(1)

portugal_dict = {}
#create a dictionary to hold data gathered from the api
portugal_dict = {
    "track_id": portugal_id_num,
    "danceability": portugal_danceability,
    "energy": portugal_energy,
    "key": portugal_key,
    "loudness": portugal_loudness,
    "mode": portugal_mode,
    "speechiness": portugal_speechiness,
    "acousticness": portugal_acousticness,
    "instrumentalness": portugal_instrumentalness,
    "liveness": portugal_liveness,
    "valence": portugal_valence,
    "tempo": portugal_tempo,
    "duration_mins": portugal_duration_mins,
    "time_signature": portugal_time_signature
}

#convert dictionary to a dataframe
portugal_info_df = pd.DataFrame(portugal_dict)

portugal_tracks = d['portugal']
portugal_df = portugal_tracks.merge(portugal_info_df, how="inner")
portugal_df.to_csv('resources/top_chart_features/portugal_df.csv', index=False)

# Romania Top Chart Track Features

In [75]:
#empty lists to store song data
romania_danceability = []
romania_energy = []
romania_key = []
romania_loudness = []
romania_mode = []
romania_speechiness = []
romania_acousticness = []
romania_instrumentalness = []
romania_liveness = []
romania_valence = []
romania_tempo = []
romania_id_num = []
romania_duration_mins = []
romania_time_signature = []

for i in d['romania']["track_id"]:
    romania_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    romania_danceability.append(romania_response["danceability"])
    romania_energy.append(romania_response["energy"])
    romania_key.append(romania_response["key"])
    romania_loudness.append(romania_response["loudness"])
    romania_mode.append(romania_response["mode"])
    romania_speechiness.append(romania_response["speechiness"])
    romania_acousticness.append(romania_response["acousticness"])
    romania_instrumentalness.append(romania_response["instrumentalness"])
    romania_liveness.append(romania_response["liveness"])
    romania_valence.append(romania_response["valence"])
    romania_tempo.append(romania_response["tempo"])
    romania_id_num.append(romania_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    romania_duration_mins.append(round((romania_response["duration_ms"] / 60000),2))
    romania_time_signature.append(romania_response["time_signature"])  

    # time.sleep(1)

romania_dict = {}
#create a dictionary to hold data gathered from the api
romania_dict = {
    "track_id": romania_id_num,
    "danceability": romania_danceability,
    "energy": romania_energy,
    "key": romania_key,
    "loudness": romania_loudness,
    "mode": romania_mode,
    "speechiness": romania_speechiness,
    "acousticness": romania_acousticness,
    "instrumentalness": romania_instrumentalness,
    "liveness": romania_liveness,
    "valence": romania_valence,
    "tempo": romania_tempo,
    "duration_mins": romania_duration_mins,
    "time_signature": romania_time_signature
}

#convert dictionary to a dataframe
romania_info_df = pd.DataFrame(romania_dict)

romania_tracks = d['romania']
romania_df = romania_tracks.merge(romania_info_df, how="inner")
romania_df.to_csv('resources/top_chart_features/romania_df.csv', index=False)

# Saudi Arabia Top Chart Track Features

In [76]:
#empty lists to store song data
saudi_arabia_danceability = []
saudi_arabia_energy = []
saudi_arabia_key = []
saudi_arabia_loudness = []
saudi_arabia_mode = []
saudi_arabia_speechiness = []
saudi_arabia_acousticness = []
saudi_arabia_instrumentalness = []
saudi_arabia_liveness = []
saudi_arabia_valence = []
saudi_arabia_tempo = []
saudi_arabia_id_num = []
saudi_arabia_duration_mins = []
saudi_arabia_time_signature = []

for i in d['saudi_arabia']["track_id"]:
    saudi_arabia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    saudi_arabia_danceability.append(saudi_arabia_response["danceability"])
    saudi_arabia_energy.append(saudi_arabia_response["energy"])
    saudi_arabia_key.append(saudi_arabia_response["key"])
    saudi_arabia_loudness.append(saudi_arabia_response["loudness"])
    saudi_arabia_mode.append(saudi_arabia_response["mode"])
    saudi_arabia_speechiness.append(saudi_arabia_response["speechiness"])
    saudi_arabia_acousticness.append(saudi_arabia_response["acousticness"])
    saudi_arabia_instrumentalness.append(saudi_arabia_response["instrumentalness"])
    saudi_arabia_liveness.append(saudi_arabia_response["liveness"])
    saudi_arabia_valence.append(saudi_arabia_response["valence"])
    saudi_arabia_tempo.append(saudi_arabia_response["tempo"])
    saudi_arabia_id_num.append(saudi_arabia_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    saudi_arabia_duration_mins.append(round((saudi_arabia_response["duration_ms"] / 60000),2))
    saudi_arabia_time_signature.append(saudi_arabia_response["time_signature"])  

    # time.sleep(1)

saudi_arabia_dict = {}
#create a dictionary to hold data gathered from the api
saudi_arabia_dict = {
    "track_id": saudi_arabia_id_num,
    "danceability": saudi_arabia_danceability,
    "energy": saudi_arabia_energy,
    "key": saudi_arabia_key,
    "loudness": saudi_arabia_loudness,
    "mode": saudi_arabia_mode,
    "speechiness": saudi_arabia_speechiness,
    "acousticness": saudi_arabia_acousticness,
    "instrumentalness": saudi_arabia_instrumentalness,
    "liveness": saudi_arabia_liveness,
    "valence": saudi_arabia_valence,
    "tempo": saudi_arabia_tempo,
    "duration_mins": saudi_arabia_duration_mins,
    "time_signature": saudi_arabia_time_signature,
}

#convert dictionary to a dataframe
saudi_arabia_info_df = pd.DataFrame(saudi_arabia_dict)

saudi_arabia_tracks = d['saudi_arabia']
saudi_arabia_df = saudi_arabia_tracks.merge(saudi_arabia_info_df, how="inner")
saudi_arabia_df['region'] = "Saudi Arabia"
saudi_arabia_df.to_csv('resources/top_chart_features/saudi_arabia_df.csv', index=False)

# Singapore Top Chart Track Features

In [77]:
#empty lists to store song data
singapore_danceability = []
singapore_energy = []
singapore_key = []
singapore_loudness = []
singapore_mode = []
singapore_speechiness = []
singapore_acousticness = []
singapore_instrumentalness = []
singapore_liveness = []
singapore_valence = []
singapore_tempo = []
singapore_id_num = []
singapore_duration_mins = []
singapore_time_signature = []

for i in d['singapore']["track_id"]:
    singapore_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    singapore_danceability.append(singapore_response["danceability"])
    singapore_energy.append(singapore_response["energy"])
    singapore_key.append(singapore_response["key"])
    singapore_loudness.append(singapore_response["loudness"])
    singapore_mode.append(singapore_response["mode"])
    singapore_speechiness.append(singapore_response["speechiness"])
    singapore_acousticness.append(singapore_response["acousticness"])
    singapore_instrumentalness.append(singapore_response["instrumentalness"])
    singapore_liveness.append(singapore_response["liveness"])
    singapore_valence.append(singapore_response["valence"])
    singapore_tempo.append(singapore_response["tempo"])
    singapore_id_num.append(singapore_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    singapore_duration_mins.append(round((singapore_response["duration_ms"] / 60000),2))
    singapore_time_signature.append(singapore_response["time_signature"])  

    # time.sleep(1)

singapore_dict = {}
#create a dictionary to hold data gathered from the api
singapore_dict = {
    "track_id": singapore_id_num,
    "danceability": singapore_danceability,
    "energy": singapore_energy,
    "key": singapore_key,
    "loudness": singapore_loudness,
    "mode": singapore_mode,
    "speechiness": singapore_speechiness,
    "acousticness": singapore_acousticness,
    "instrumentalness": singapore_instrumentalness,
    "liveness": singapore_liveness,
    "valence": singapore_valence,
    "tempo": singapore_tempo,
    "duration_mins": singapore_duration_mins,
    "time_signature": singapore_time_signature
}

#convert dictionary to a dataframe
singapore_info_df = pd.DataFrame(singapore_dict)

singapore_tracks = d['singapore']
singapore_df = singapore_tracks.merge(singapore_info_df, how="inner")
singapore_df.to_csv('resources/top_chart_features/singapore_df.csv', index=False)

# Slovakia Top Chart Track Features

In [78]:
#empty lists to store song data
slovakia_danceability = []
slovakia_energy = []
slovakia_key = []
slovakia_loudness = []
slovakia_mode = []
slovakia_speechiness = []
slovakia_acousticness = []
slovakia_instrumentalness = []
slovakia_liveness = []
slovakia_valence = []
slovakia_tempo = []
slovakia_id_num = []
slovakia_duration_mins = []
slovakia_time_signature = []

for i in d['slovakia']["track_id"]:
    slovakia_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    slovakia_danceability.append(slovakia_response["danceability"])
    slovakia_energy.append(slovakia_response["energy"])
    slovakia_key.append(slovakia_response["key"])
    slovakia_loudness.append(slovakia_response["loudness"])
    slovakia_mode.append(slovakia_response["mode"])
    slovakia_speechiness.append(slovakia_response["speechiness"])
    slovakia_acousticness.append(slovakia_response["acousticness"])
    slovakia_instrumentalness.append(slovakia_response["instrumentalness"])
    slovakia_liveness.append(slovakia_response["liveness"])
    slovakia_valence.append(slovakia_response["valence"])
    slovakia_tempo.append(slovakia_response["tempo"])
    slovakia_id_num.append(slovakia_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    slovakia_duration_mins.append(round((slovakia_response["duration_ms"] / 60000),2))
    slovakia_time_signature.append(slovakia_response["time_signature"])  

    # time.sleep(1)

slovakia_dict = {}
#create a dictionary to hold data gathered from the api
slovakia_dict = {
    "track_id": slovakia_id_num,
    "danceability": slovakia_danceability,
    "energy": slovakia_energy,
    "key": slovakia_key,
    "loudness": slovakia_loudness,
    "mode": slovakia_mode,
    "speechiness": slovakia_speechiness,
    "acousticness": slovakia_acousticness,
    "instrumentalness": slovakia_instrumentalness,
    "liveness": slovakia_liveness,
    "valence": slovakia_valence,
    "tempo": slovakia_tempo,
    "duration_mins": slovakia_duration_mins,
    "time_signature": slovakia_time_signature
}

#convert dictionary to a dataframe
slovakia_info_df = pd.DataFrame(slovakia_dict)

slovakia_tracks = d['slovakia']
slovakia_df = slovakia_tracks.merge(slovakia_info_df, how="inner")
slovakia_df.to_csv('resources/top_chart_features/slovakia_df.csv', index=False)

# South Africa Top Chart Track Features

In [82]:
#empty lists to store song data
south_africa_danceability = []
south_africa_energy = []
south_africa_key = []
south_africa_loudness = []
south_africa_mode = []
south_africa_speechiness = []
south_africa_acousticness = []
south_africa_instrumentalness = []
south_africa_liveness = []
south_africa_valence = []
south_africa_tempo = []
south_africa_id_num = []
south_africa_duration_mins = []
south_africa_time_signature = []

for i in d['south_africa']["track_id"]:
    south_africa_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    south_africa_danceability.append(south_africa_response["danceability"])
    south_africa_energy.append(south_africa_response["energy"])
    south_africa_key.append(south_africa_response["key"])
    south_africa_loudness.append(south_africa_response["loudness"])
    south_africa_mode.append(south_africa_response["mode"])
    south_africa_speechiness.append(south_africa_response["speechiness"])
    south_africa_acousticness.append(south_africa_response["acousticness"])
    south_africa_instrumentalness.append(south_africa_response["instrumentalness"])
    south_africa_liveness.append(south_africa_response["liveness"])
    south_africa_valence.append(south_africa_response["valence"])
    south_africa_tempo.append(south_africa_response["tempo"])
    south_africa_id_num.append(south_africa_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    south_africa_duration_mins.append(round((south_africa_response["duration_ms"] / 60000),2))
    south_africa_time_signature.append(south_africa_response["time_signature"])  

    # time.sleep(1)

south_africa_dict = {}
#create a dictionary to hold data gathered from the api
south_africa_dict = {
    "track_id": south_africa_id_num,
    "danceability": south_africa_danceability,
    "energy": south_africa_energy,
    "key": south_africa_key,
    "loudness": south_africa_loudness,
    "mode": south_africa_mode,
    "speechiness": south_africa_speechiness,
    "acousticness": south_africa_acousticness,
    "instrumentalness": south_africa_instrumentalness,
    "liveness": south_africa_liveness,
    "valence": south_africa_valence,
    "tempo": south_africa_tempo,
    "duration_mins": south_africa_duration_mins,
    "time_signature": south_africa_time_signature,
}

#convert dictionary to a dataframe
south_africa_info_df = pd.DataFrame(south_africa_dict)

south_africa_tracks = d['south_africa']
south_africa_df = south_africa_tracks.merge(south_africa_info_df, how="inner")
south_africa_df['region'] = "South Africa"
south_africa_df.to_csv('resources/top_chart_features/south_africa_df.csv', index=False)

# South Korea Top Chart Track Features

In [83]:
#empty lists to store song data
south_korea_danceability = []
south_korea_energy = []
south_korea_key = []
south_korea_loudness = []
south_korea_mode = []
south_korea_speechiness = []
south_korea_acousticness = []
south_korea_instrumentalness = []
south_korea_liveness = []
south_korea_valence = []
south_korea_tempo = []
south_korea_id_num = []
south_korea_duration_mins = []
south_korea_time_signature = []

for i in d['south_korea']["track_id"]:
    south_korea_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    south_korea_danceability.append(south_korea_response["danceability"])
    south_korea_energy.append(south_korea_response["energy"])
    south_korea_key.append(south_korea_response["key"])
    south_korea_loudness.append(south_korea_response["loudness"])
    south_korea_mode.append(south_korea_response["mode"])
    south_korea_speechiness.append(south_korea_response["speechiness"])
    south_korea_acousticness.append(south_korea_response["acousticness"])
    south_korea_instrumentalness.append(south_korea_response["instrumentalness"])
    south_korea_liveness.append(south_korea_response["liveness"])
    south_korea_valence.append(south_korea_response["valence"])
    south_korea_tempo.append(south_korea_response["tempo"])
    south_korea_id_num.append(south_korea_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    south_korea_duration_mins.append(round((south_korea_response["duration_ms"] / 60000),2))
    south_korea_time_signature.append(south_korea_response["time_signature"])  

    # time.sleep(1)

south_korea_dict = {}
#create a dictionary to hold data gathered from the api
south_korea_dict = {
    "track_id": south_korea_id_num,
    "danceability": south_korea_danceability,
    "energy": south_korea_energy,
    "key": south_korea_key,
    "loudness": south_korea_loudness,
    "mode": south_korea_mode,
    "speechiness": south_korea_speechiness,
    "acousticness": south_korea_acousticness,
    "instrumentalness": south_korea_instrumentalness,
    "liveness": south_korea_liveness,
    "valence": south_korea_valence,
    "tempo": south_korea_tempo,
    "duration_mins": south_korea_duration_mins,
    "time_signature": south_korea_time_signature,
}

#convert dictionary to a dataframe
south_korea_info_df = pd.DataFrame(south_korea_dict)

south_korea_tracks = d['south_korea']
south_korea_df = south_korea_tracks.merge(south_korea_info_df, how="inner")
south_korea_df['region'] = "South Korea"
south_korea_df.to_csv('resources/top_chart_features/south_korea_df.csv', index=False)

# Spain Top Chart Track Features

In [84]:
#empty lists to store song data
spain_danceability = []
spain_energy = []
spain_key = []
spain_loudness = []
spain_mode = []
spain_speechiness = []
spain_acousticness = []
spain_instrumentalness = []
spain_liveness = []
spain_valence = []
spain_tempo = []
spain_id_num = []
spain_duration_mins = []
spain_time_signature = []

for i in d['spain']["track_id"]:
    spain_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    spain_danceability.append(spain_response["danceability"])
    spain_energy.append(spain_response["energy"])
    spain_key.append(spain_response["key"])
    spain_loudness.append(spain_response["loudness"])
    spain_mode.append(spain_response["mode"])
    spain_speechiness.append(spain_response["speechiness"])
    spain_acousticness.append(spain_response["acousticness"])
    spain_instrumentalness.append(spain_response["instrumentalness"])
    spain_liveness.append(spain_response["liveness"])
    spain_valence.append(spain_response["valence"])
    spain_tempo.append(spain_response["tempo"])
    spain_id_num.append(spain_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    spain_duration_mins.append(round((spain_response["duration_ms"] / 60000),2))
    spain_time_signature.append(spain_response["time_signature"])  

    # time.sleep(1)

spain_dict = {}
#create a dictionary to hold data gathered from the api
spain_dict = {
    "track_id": spain_id_num,
    "danceability": spain_danceability,
    "energy": spain_energy,
    "key": spain_key,
    "loudness": spain_loudness,
    "mode": spain_mode,
    "speechiness": spain_speechiness,
    "acousticness": spain_acousticness,
    "instrumentalness": spain_instrumentalness,
    "liveness": spain_liveness,
    "valence": spain_valence,
    "tempo": spain_tempo,
    "duration_mins": spain_duration_mins,
    "time_signature": spain_time_signature
}

#convert dictionary to a dataframe
spain_info_df = pd.DataFrame(spain_dict)

spain_tracks = d['spain']
spain_df = spain_tracks.merge(spain_info_df, how="inner")
spain_df.to_csv('resources/top_chart_features/spain_df.csv', index=False)

# Sweden Top Chart Track Features

In [86]:
#empty lists to store song data
sweden_danceability = []
sweden_energy = []
sweden_key = []
sweden_loudness = []
sweden_mode = []
sweden_speechiness = []
sweden_acousticness = []
sweden_instrumentalness = []
sweden_liveness = []
sweden_valence = []
sweden_tempo = []
sweden_id_num = []
sweden_duration_mins = []
sweden_time_signature = []

for i in d['sweden']["track_id"]:
    sweden_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    sweden_danceability.append(sweden_response["danceability"])
    sweden_energy.append(sweden_response["energy"])
    sweden_key.append(sweden_response["key"])
    sweden_loudness.append(sweden_response["loudness"])
    sweden_mode.append(sweden_response["mode"])
    sweden_speechiness.append(sweden_response["speechiness"])
    sweden_acousticness.append(sweden_response["acousticness"])
    sweden_instrumentalness.append(sweden_response["instrumentalness"])
    sweden_liveness.append(sweden_response["liveness"])
    sweden_valence.append(sweden_response["valence"])
    sweden_tempo.append(sweden_response["tempo"])
    sweden_id_num.append(sweden_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    sweden_duration_mins.append(round((sweden_response["duration_ms"] / 60000),2))
    sweden_time_signature.append(sweden_response["time_signature"])  

    # time.sleep(1)

sweden_dict = {}
#create a dictionary to hold data gathered from the api
sweden_dict = {
    "track_id": sweden_id_num,
    "danceability": sweden_danceability,
    "energy": sweden_energy,
    "key": sweden_key,
    "loudness": sweden_loudness,
    "mode": sweden_mode,
    "speechiness": sweden_speechiness,
    "acousticness": sweden_acousticness,
    "instrumentalness": sweden_instrumentalness,
    "liveness": sweden_liveness,
    "valence": sweden_valence,
    "tempo": sweden_tempo,
    "duration_mins": sweden_duration_mins,
    "time_signature": sweden_time_signature
}

#convert dictionary to a dataframe
sweden_info_df = pd.DataFrame(sweden_dict)

sweden_tracks = d['sweden']
sweden_df = sweden_tracks.merge(sweden_info_df, how="inner")
sweden_df.to_csv('resources/top_chart_features/sweden_df.csv', index=False)

# Switzerland Top Chart Track Features

In [87]:
#empty lists to store song data
switzerland_danceability = []
switzerland_energy = []
switzerland_key = []
switzerland_loudness = []
switzerland_mode = []
switzerland_speechiness = []
switzerland_acousticness = []
switzerland_instrumentalness = []
switzerland_liveness = []
switzerland_valence = []
switzerland_tempo = []
switzerland_id_num = []
switzerland_duration_mins = []
switzerland_time_signature = []

for i in d['switzerland']["track_id"]:
    switzerland_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    switzerland_danceability.append(switzerland_response["danceability"])
    switzerland_energy.append(switzerland_response["energy"])
    switzerland_key.append(switzerland_response["key"])
    switzerland_loudness.append(switzerland_response["loudness"])
    switzerland_mode.append(switzerland_response["mode"])
    switzerland_speechiness.append(switzerland_response["speechiness"])
    switzerland_acousticness.append(switzerland_response["acousticness"])
    switzerland_instrumentalness.append(switzerland_response["instrumentalness"])
    switzerland_liveness.append(switzerland_response["liveness"])
    switzerland_valence.append(switzerland_response["valence"])
    switzerland_tempo.append(switzerland_response["tempo"])
    switzerland_id_num.append(switzerland_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    switzerland_duration_mins.append(round((switzerland_response["duration_ms"] / 60000),2))
    switzerland_time_signature.append(switzerland_response["time_signature"])  

    # time.sleep(1)

switzerland_dict = {}
#create a dictionary to hold data gathered from the api
switzerland_dict = {
    "track_id": switzerland_id_num,
    "danceability": switzerland_danceability,
    "energy": switzerland_energy,
    "key": switzerland_key,
    "loudness": switzerland_loudness,
    "mode": switzerland_mode,
    "speechiness": switzerland_speechiness,
    "acousticness": switzerland_acousticness,
    "instrumentalness": switzerland_instrumentalness,
    "liveness": switzerland_liveness,
    "valence": switzerland_valence,
    "tempo": switzerland_tempo,
    "duration_mins": switzerland_duration_mins,
    "time_signature": switzerland_time_signature
}

#convert dictionary to a dataframe
switzerland_info_df = pd.DataFrame(switzerland_dict)

switzerland_tracks = d['switzerland']
switzerland_df = switzerland_tracks.merge(switzerland_info_df, how="inner")
switzerland_df.to_csv('resources/top_chart_features/switzerland_df.csv', index=False)

# Taiwan Top Chart Track Features

In [88]:
#empty lists to store song data
taiwan_danceability = []
taiwan_energy = []
taiwan_key = []
taiwan_loudness = []
taiwan_mode = []
taiwan_speechiness = []
taiwan_acousticness = []
taiwan_instrumentalness = []
taiwan_liveness = []
taiwan_valence = []
taiwan_tempo = []
taiwan_id_num = []
taiwan_duration_mins = []
taiwan_time_signature = []

for i in d['taiwan']["track_id"]:
    taiwan_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    taiwan_danceability.append(taiwan_response["danceability"])
    taiwan_energy.append(taiwan_response["energy"])
    taiwan_key.append(taiwan_response["key"])
    taiwan_loudness.append(taiwan_response["loudness"])
    taiwan_mode.append(taiwan_response["mode"])
    taiwan_speechiness.append(taiwan_response["speechiness"])
    taiwan_acousticness.append(taiwan_response["acousticness"])
    taiwan_instrumentalness.append(taiwan_response["instrumentalness"])
    taiwan_liveness.append(taiwan_response["liveness"])
    taiwan_valence.append(taiwan_response["valence"])
    taiwan_tempo.append(taiwan_response["tempo"])
    taiwan_id_num.append(taiwan_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    taiwan_duration_mins.append(round((taiwan_response["duration_ms"] / 60000),2))
    taiwan_time_signature.append(taiwan_response["time_signature"])  

    # time.sleep(1)

taiwan_dict = {}
#create a dictionary to hold data gathered from the api
taiwan_dict = {
    "track_id": taiwan_id_num,
    "danceability": taiwan_danceability,
    "energy": taiwan_energy,
    "key": taiwan_key,
    "loudness": taiwan_loudness,
    "mode": taiwan_mode,
    "speechiness": taiwan_speechiness,
    "acousticness": taiwan_acousticness,
    "instrumentalness": taiwan_instrumentalness,
    "liveness": taiwan_liveness,
    "valence": taiwan_valence,
    "tempo": taiwan_tempo,
    "duration_mins": taiwan_duration_mins,
    "time_signature": taiwan_time_signature
}

#convert dictionary to a dataframe
taiwan_info_df = pd.DataFrame(taiwan_dict)

taiwan_tracks = d['taiwan']
taiwan_df = taiwan_tracks.merge(taiwan_info_df, how="inner")
taiwan_df.to_csv('resources/top_chart_features/taiwan_df.csv', index=False)

# Thailand Top Chart Track Features

In [89]:
#empty lists to store song data
thailand_danceability = []
thailand_energy = []
thailand_key = []
thailand_loudness = []
thailand_mode = []
thailand_speechiness = []
thailand_acousticness = []
thailand_instrumentalness = []
thailand_liveness = []
thailand_valence = []
thailand_tempo = []
thailand_id_num = []
thailand_duration_mins = []
thailand_time_signature = []

for i in d['thailand']["track_id"]:
    thailand_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    thailand_danceability.append(thailand_response["danceability"])
    thailand_energy.append(thailand_response["energy"])
    thailand_key.append(thailand_response["key"])
    thailand_loudness.append(thailand_response["loudness"])
    thailand_mode.append(thailand_response["mode"])
    thailand_speechiness.append(thailand_response["speechiness"])
    thailand_acousticness.append(thailand_response["acousticness"])
    thailand_instrumentalness.append(thailand_response["instrumentalness"])
    thailand_liveness.append(thailand_response["liveness"])
    thailand_valence.append(thailand_response["valence"])
    thailand_tempo.append(thailand_response["tempo"])
    thailand_id_num.append(thailand_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    thailand_duration_mins.append(round((thailand_response["duration_ms"] / 60000),2))
    thailand_time_signature.append(thailand_response["time_signature"])  

    # time.sleep(1)

thailand_dict = {}
#create a dictionary to hold data gathered from the api
thailand_dict = {
    "track_id": thailand_id_num,
    "danceability": thailand_danceability,
    "energy": thailand_energy,
    "key": thailand_key,
    "loudness": thailand_loudness,
    "mode": thailand_mode,
    "speechiness": thailand_speechiness,
    "acousticness": thailand_acousticness,
    "instrumentalness": thailand_instrumentalness,
    "liveness": thailand_liveness,
    "valence": thailand_valence,
    "tempo": thailand_tempo,
    "duration_mins": thailand_duration_mins,
    "time_signature": thailand_time_signature
}

#convert dictionary to a dataframe
thailand_info_df = pd.DataFrame(thailand_dict)

thailand_tracks = d['thailand']
thailand_df = thailand_tracks.merge(thailand_info_df, how="inner")
thailand_df.to_csv('resources/top_chart_features/thailand_df.csv', index=False)

# Turkey Top Chart Track Features

In [90]:
#empty lists to store song data
turkey_danceability = []
turkey_energy = []
turkey_key = []
turkey_loudness = []
turkey_mode = []
turkey_speechiness = []
turkey_acousticness = []
turkey_instrumentalness = []
turkey_liveness = []
turkey_valence = []
turkey_tempo = []
turkey_id_num = []
turkey_duration_mins = []
turkey_time_signature = []

for i in d['turkey']["track_id"]:
    turkey_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    turkey_danceability.append(turkey_response["danceability"])
    turkey_energy.append(turkey_response["energy"])
    turkey_key.append(turkey_response["key"])
    turkey_loudness.append(turkey_response["loudness"])
    turkey_mode.append(turkey_response["mode"])
    turkey_speechiness.append(turkey_response["speechiness"])
    turkey_acousticness.append(turkey_response["acousticness"])
    turkey_instrumentalness.append(turkey_response["instrumentalness"])
    turkey_liveness.append(turkey_response["liveness"])
    turkey_valence.append(turkey_response["valence"])
    turkey_tempo.append(turkey_response["tempo"])
    turkey_id_num.append(turkey_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    turkey_duration_mins.append(round((turkey_response["duration_ms"] / 60000),2))
    turkey_time_signature.append(turkey_response["time_signature"])  

    # time.sleep(1)

turkey_dict = {}
#create a dictionary to hold data gathered from the api
turkey_dict = {
    "track_id": turkey_id_num,
    "danceability": turkey_danceability,
    "energy": turkey_energy,
    "key": turkey_key,
    "loudness": turkey_loudness,
    "mode": turkey_mode,
    "speechiness": turkey_speechiness,
    "acousticness": turkey_acousticness,
    "instrumentalness": turkey_instrumentalness,
    "liveness": turkey_liveness,
    "valence": turkey_valence,
    "tempo": turkey_tempo,
    "duration_mins": turkey_duration_mins,
    "time_signature": turkey_time_signature
}

#convert dictionary to a dataframe
turkey_info_df = pd.DataFrame(turkey_dict)

turkey_tracks = d['turkey']
turkey_df = turkey_tracks.merge(turkey_info_df, how="inner")
turkey_df.to_csv('resources/top_chart_features/turkey_df.csv', index=False)

# UAE Top Chart Track Features

In [91]:
#empty lists to store song data
uae_danceability = []
uae_energy = []
uae_key = []
uae_loudness = []
uae_mode = []
uae_speechiness = []
uae_acousticness = []
uae_instrumentalness = []
uae_liveness = []
uae_valence = []
uae_tempo = []
uae_id_num = []
uae_duration_mins = []
uae_time_signature = []

for i in d['uae']["track_id"]:
    uae_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    uae_danceability.append(uae_response["danceability"])
    uae_energy.append(uae_response["energy"])
    uae_key.append(uae_response["key"])
    uae_loudness.append(uae_response["loudness"])
    uae_mode.append(uae_response["mode"])
    uae_speechiness.append(uae_response["speechiness"])
    uae_acousticness.append(uae_response["acousticness"])
    uae_instrumentalness.append(uae_response["instrumentalness"])
    uae_liveness.append(uae_response["liveness"])
    uae_valence.append(uae_response["valence"])
    uae_tempo.append(uae_response["tempo"])
    uae_id_num.append(uae_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    uae_duration_mins.append(round((uae_response["duration_ms"] / 60000),2))
    uae_time_signature.append(uae_response["time_signature"])  

    # time.sleep(1)

uae_dict = {}
#create a dictionary to hold data gathered from the api
uae_dict = {
    "track_id": uae_id_num,
    "danceability": uae_danceability,
    "energy": uae_energy,
    "key": uae_key,
    "loudness": uae_loudness,
    "mode": uae_mode,
    "speechiness": uae_speechiness,
    "acousticness": uae_acousticness,
    "instrumentalness": uae_instrumentalness,
    "liveness": uae_liveness,
    "valence": uae_valence,
    "tempo": uae_tempo,
    "duration_mins": uae_duration_mins,
    "time_signature": uae_time_signature
}

#convert dictionary to a dataframe
uae_info_df = pd.DataFrame(uae_dict)

uae_tracks = d['uae']
uae_df = uae_tracks.merge(uae_info_df, how="inner")
uae_df.to_csv('resources/top_chart_features/uae_df.csv', index=False)

# Ukraine Top Chart Track Features

In [92]:
#empty lists to store song data
ukraine_danceability = []
ukraine_energy = []
ukraine_key = []
ukraine_loudness = []
ukraine_mode = []
ukraine_speechiness = []
ukraine_acousticness = []
ukraine_instrumentalness = []
ukraine_liveness = []
ukraine_valence = []
ukraine_tempo = []
ukraine_id_num = []
ukraine_duration_mins = []
ukraine_time_signature = []

for i in d['ukraine']["track_id"]:
    ukraine_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    ukraine_danceability.append(ukraine_response["danceability"])
    ukraine_energy.append(ukraine_response["energy"])
    ukraine_key.append(ukraine_response["key"])
    ukraine_loudness.append(ukraine_response["loudness"])
    ukraine_mode.append(ukraine_response["mode"])
    ukraine_speechiness.append(ukraine_response["speechiness"])
    ukraine_acousticness.append(ukraine_response["acousticness"])
    ukraine_instrumentalness.append(ukraine_response["instrumentalness"])
    ukraine_liveness.append(ukraine_response["liveness"])
    ukraine_valence.append(ukraine_response["valence"])
    ukraine_tempo.append(ukraine_response["tempo"])
    ukraine_id_num.append(ukraine_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    ukraine_duration_mins.append(round((ukraine_response["duration_ms"] / 60000),2))
    ukraine_time_signature.append(ukraine_response["time_signature"])  

    # time.sleep(1)

ukraine_dict = {}
#create a dictionary to hold data gathered from the api
ukraine_dict = {
    "track_id": ukraine_id_num,
    "danceability": ukraine_danceability,
    "energy": ukraine_energy,
    "key": ukraine_key,
    "loudness": ukraine_loudness,
    "mode": ukraine_mode,
    "speechiness": ukraine_speechiness,
    "acousticness": ukraine_acousticness,
    "instrumentalness": ukraine_instrumentalness,
    "liveness": ukraine_liveness,
    "valence": ukraine_valence,
    "tempo": ukraine_tempo,
    "duration_mins": ukraine_duration_mins,
    "time_signature": ukraine_time_signature
}

#convert dictionary to a dataframe
ukraine_info_df = pd.DataFrame(ukraine_dict)

ukraine_tracks = d['ukraine']
ukraine_df = ukraine_tracks.merge(ukraine_info_df, how="inner")
ukraine_df.to_csv('resources/top_chart_features/ukraine_df.csv', index=False)

# United Kingdom Top Chart Track Features

In [93]:
#empty lists to store song data
united_kingdom_danceability = []
united_kingdom_energy = []
united_kingdom_key = []
united_kingdom_loudness = []
united_kingdom_mode = []
united_kingdom_speechiness = []
united_kingdom_acousticness = []
united_kingdom_instrumentalness = []
united_kingdom_liveness = []
united_kingdom_valence = []
united_kingdom_tempo = []
united_kingdom_id_num = []
united_kingdom_duration_mins = []
united_kingdom_time_signature = []

for i in d['united_kingdom']["track_id"]:
    united_kingdom_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    united_kingdom_danceability.append(united_kingdom_response["danceability"])
    united_kingdom_energy.append(united_kingdom_response["energy"])
    united_kingdom_key.append(united_kingdom_response["key"])
    united_kingdom_loudness.append(united_kingdom_response["loudness"])
    united_kingdom_mode.append(united_kingdom_response["mode"])
    united_kingdom_speechiness.append(united_kingdom_response["speechiness"])
    united_kingdom_acousticness.append(united_kingdom_response["acousticness"])
    united_kingdom_instrumentalness.append(united_kingdom_response["instrumentalness"])
    united_kingdom_liveness.append(united_kingdom_response["liveness"])
    united_kingdom_valence.append(united_kingdom_response["valence"])
    united_kingdom_tempo.append(united_kingdom_response["tempo"])
    united_kingdom_id_num.append(united_kingdom_response["id"])
    #duration recorded in api in milliseconds, converting to minutes here
    united_kingdom_duration_mins.append(round((united_kingdom_response["duration_ms"] / 60000),2))
    united_kingdom_time_signature.append(united_kingdom_response["time_signature"])  

    # time.sleep(1)

united_kingdom_dict = {}
#create a dictionary to hold data gathered from the api
united_kingdom_dict = {
    "track_id": united_kingdom_id_num,
    "danceability": united_kingdom_danceability,
    "energy": united_kingdom_energy,
    "key": united_kingdom_key,
    "loudness": united_kingdom_loudness,
    "mode": united_kingdom_mode,
    "speechiness": united_kingdom_speechiness,
    "acousticness": united_kingdom_acousticness,
    "instrumentalness": united_kingdom_instrumentalness,
    "liveness": united_kingdom_liveness,
    "valence": united_kingdom_valence,
    "tempo": united_kingdom_tempo,
    "duration_mins": united_kingdom_duration_mins,
    "time_signature": united_kingdom_time_signature,
}

#convert dictionary to a dataframe
united_kingdom_info_df = pd.DataFrame(united_kingdom_dict)

united_kingdom_tracks = d['united_kingdom']
united_kingdom_df = united_kingdom_tracks.merge(united_kingdom_info_df, how="inner")
united_kingdom_df['region'] = "United Kingdom"
united_kingdom_df.to_csv('resources/top_chart_features/united_kingdom_df.csv', index=False)

# Uruguay Top Chart Track Features

In [94]:
#empty lists to store song data
uruguay_danceability = []
uruguay_energy = []
uruguay_key = []
uruguay_loudness = []
uruguay_mode = []
uruguay_speechiness = []
uruguay_acousticness = []
uruguay_instrumentalness = []
uruguay_liveness = []
uruguay_valence = []
uruguay_tempo = []
uruguay_id_num = []
uruguay_duration_mins = []
uruguay_time_signature = []

for i in d['uruguay']["track_id"]:
    uruguay_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    uruguay_danceability.append(uruguay_response["danceability"])
    uruguay_energy.append(uruguay_response["energy"])
    uruguay_key.append(uruguay_response["key"])
    uruguay_loudness.append(uruguay_response["loudness"])
    uruguay_mode.append(uruguay_response["mode"])
    uruguay_speechiness.append(uruguay_response["speechiness"])
    uruguay_acousticness.append(uruguay_response["acousticness"])
    uruguay_instrumentalness.append(uruguay_response["instrumentalness"])
    uruguay_liveness.append(uruguay_response["liveness"])
    uruguay_valence.append(uruguay_response["valence"])
    uruguay_tempo.append(uruguay_response["tempo"])
    uruguay_id_num.append(uruguay_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    uruguay_duration_mins.append(round((uruguay_response["duration_ms"] / 60000),2))
    uruguay_time_signature.append(uruguay_response["time_signature"])  

    # time.sleep(1)

uruguay_dict = {}
#create a dictionary to hold data gathered from the api
uruguay_dict = {
    "track_id": uruguay_id_num,
    "danceability": uruguay_danceability,
    "energy": uruguay_energy,
    "key": uruguay_key,
    "loudness": uruguay_loudness,
    "mode": uruguay_mode,
    "speechiness": uruguay_speechiness,
    "acousticness": uruguay_acousticness,
    "instrumentalness": uruguay_instrumentalness,
    "liveness": uruguay_liveness,
    "valence": uruguay_valence,
    "tempo": uruguay_tempo,
    "duration_mins": uruguay_duration_mins,
    "time_signature": uruguay_time_signature
}

#convert dictionary to a dataframe
uruguay_info_df = pd.DataFrame(uruguay_dict)

uruguay_tracks = d['uruguay']
uruguay_df = uruguay_tracks.merge(uruguay_info_df, how="inner")
uruguay_df.to_csv('resources/top_chart_features/uruguay_df.csv', index=False)

# Venezuela Top Chart Track Features

In [95]:
#empty lists to store song data
venezuela_danceability = []
venezuela_energy = []
venezuela_key = []
venezuela_loudness = []
venezuela_mode = []
venezuela_speechiness = []
venezuela_acousticness = []
venezuela_instrumentalness = []
venezuela_liveness = []
venezuela_valence = []
venezuela_tempo = []
venezuela_id_num = []
venezuela_duration_mins = []
venezuela_time_signature = []

for i in d['venezuela']["track_id"]:
    venezuela_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    venezuela_danceability.append(venezuela_response["danceability"])
    venezuela_energy.append(venezuela_response["energy"])
    venezuela_key.append(venezuela_response["key"])
    venezuela_loudness.append(venezuela_response["loudness"])
    venezuela_mode.append(venezuela_response["mode"])
    venezuela_speechiness.append(venezuela_response["speechiness"])
    venezuela_acousticness.append(venezuela_response["acousticness"])
    venezuela_instrumentalness.append(venezuela_response["instrumentalness"])
    venezuela_liveness.append(venezuela_response["liveness"])
    venezuela_valence.append(venezuela_response["valence"])
    venezuela_tempo.append(venezuela_response["tempo"])
    venezuela_id_num.append(venezuela_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    venezuela_duration_mins.append(round((venezuela_response["duration_ms"] / 60000),2))
    venezuela_time_signature.append(venezuela_response["time_signature"])  

    # time.sleep(1)

venezuela_dict = {}
#create a dictionary to hold data gathered from the api
venezuela_dict = {
    "track_id": venezuela_id_num,
    "danceability": venezuela_danceability,
    "energy": venezuela_energy,
    "key": venezuela_key,
    "loudness": venezuela_loudness,
    "mode": venezuela_mode,
    "speechiness": venezuela_speechiness,
    "acousticness": venezuela_acousticness,
    "instrumentalness": venezuela_instrumentalness,
    "liveness": venezuela_liveness,
    "valence": venezuela_valence,
    "tempo": venezuela_tempo,
    "duration_mins": venezuela_duration_mins,
    "time_signature": venezuela_time_signature
}

#convert dictionary to a dataframe
venezuela_info_df = pd.DataFrame(venezuela_dict)

venezuela_tracks = d['venezuela']
venezuela_df = venezuela_tracks.merge(venezuela_info_df, how="inner")
venezuela_df.to_csv('resources/top_chart_features/venezuela_df.csv', index=False)

# Vietnam Top Chart Track Features

In [96]:
#empty lists to store song data
vietnam_danceability = []
vietnam_energy = []
vietnam_key = []
vietnam_loudness = []
vietnam_mode = []
vietnam_speechiness = []
vietnam_acousticness = []
vietnam_instrumentalness = []
vietnam_liveness = []
vietnam_valence = []
vietnam_tempo = []
vietnam_id_num = []
vietnam_duration_mins = []
vietnam_time_signature = []

for i in d['vietnam']["track_id"]:
    vietnam_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    vietnam_danceability.append(vietnam_response["danceability"])
    vietnam_energy.append(vietnam_response["energy"])
    vietnam_key.append(vietnam_response["key"])
    vietnam_loudness.append(vietnam_response["loudness"])
    vietnam_mode.append(vietnam_response["mode"])
    vietnam_speechiness.append(vietnam_response["speechiness"])
    vietnam_acousticness.append(vietnam_response["acousticness"])
    vietnam_instrumentalness.append(vietnam_response["instrumentalness"])
    vietnam_liveness.append(vietnam_response["liveness"])
    vietnam_valence.append(vietnam_response["valence"])
    vietnam_tempo.append(vietnam_response["tempo"])
    vietnam_id_num.append(vietnam_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    vietnam_duration_mins.append(round((vietnam_response["duration_ms"] / 60000),2))
    vietnam_time_signature.append(vietnam_response["time_signature"])  

    # time.sleep(1)

vietnam_dict = {}
#create a dictionary to hold data gathered from the api
vietnam_dict = {
    "track_id": vietnam_id_num,
    "danceability": vietnam_danceability,
    "energy": vietnam_energy,
    "key": vietnam_key,
    "loudness": vietnam_loudness,
    "mode": vietnam_mode,
    "speechiness": vietnam_speechiness,
    "acousticness": vietnam_acousticness,
    "instrumentalness": vietnam_instrumentalness,
    "liveness": vietnam_liveness,
    "valence": vietnam_valence,
    "tempo": vietnam_tempo,
    "duration_mins": vietnam_duration_mins,
    "time_signature": vietnam_time_signature
}

#convert dictionary to a dataframe
vietnam_info_df = pd.DataFrame(vietnam_dict)

vietnam_tracks = d['vietnam']
vietnam_df = vietnam_tracks.merge(vietnam_info_df, how="inner")
vietnam_df.to_csv('resources/top_chart_features/vietnam_df.csv', index=False)

# Global Top Chart Track Features

In [97]:
#empty lists to store song data
global_danceability = []
global_energy = []
global_key = []
global_loudness = []
global_mode = []
global_speechiness = []
global_acousticness = []
global_instrumentalness = []
global_liveness = []
global_valence = []
global_tempo = []
global_id_num = []
global_duration_mins = []
global_time_signature = []

for i in d['global']["track_id"]:
    global_response = requests.get(base_url + 'audio-features/' + i, headers=headers).json()
    
    #fill in track info for each audio feature / empty list
    global_danceability.append(global_response["danceability"])
    global_energy.append(global_response["energy"])
    global_key.append(global_response["key"])
    global_loudness.append(global_response["loudness"])
    global_mode.append(global_response["mode"])
    global_speechiness.append(global_response["speechiness"])
    global_acousticness.append(global_response["acousticness"])
    global_instrumentalness.append(global_response["instrumentalness"])
    global_liveness.append(global_response["liveness"])
    global_valence.append(global_response["valence"])
    global_tempo.append(global_response["tempo"])
    global_id_num.append(global_response["id"])  
    #duration recorded in api in milliseconds, converting to minutes here
    global_duration_mins.append(round((global_response["duration_ms"] / 60000),2))
    global_time_signature.append(global_response["time_signature"])  

    # time.sleep(1)

global_dict = {}
#create a dictionary to hold data gathered from the api
global_dict = {
    "track_id": global_id_num,
    "danceability": global_danceability,
    "energy": global_energy,
    "key": global_key,
    "loudness": global_loudness,
    "mode": global_mode,
    "speechiness": global_speechiness,
    "acousticness": global_acousticness,
    "instrumentalness": global_instrumentalness,
    "liveness": global_liveness,
    "valence": global_valence,
    "tempo": global_tempo,
    "duration_mins": global_duration_mins,
    "time_signature": global_time_signature
}

#convert dictionary to a dataframe
global_info_df = pd.DataFrame(global_dict)

global_tracks = d['global']
global_df = global_tracks.merge(global_info_df, how="inner")
global_df.to_csv('resources/top_chart_features/global_df.csv', index=False)